In [ ]:
def initializeVectorStore():
    # docs = load_docs(directory)
    # #print(docs)
    # chunk_size = 1000
    # chunk_overlap = 100
    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=chunk_size, chunk_overlap=chunk_overlap
    # )
    # split_docs = text_splitter.split_documents(docs)

    # vectorstore = PineconeVectorStore.from_documents(
    #     split_docs, embeddings, index_name=index_name
    # )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    
    
    

    return vectorstore


docs = load_docs(directory)

chunk_size = 200
chunk_overlap = 20
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
split_docs = text_splitter.split_documents(docs)

# vectorstore = PineconeVectorStore.from_documents(
#     split_docs, embeddings, index_name=index_name
# )
#vectorstore = PineconeVectorStore(index_name=index_name, embedding=OpenAIEmbeddings())

# keyword = """ """
query = " "


# retriever.get_relevant_documents(keyword)
# retriever.invoke(keyword)
# retriever



#llm = ChatOpenAI(model="gpt-4o", temperature=0.5)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)

# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstore.as_retriever(),
#     return_source_documents=True,
# )


class PineconeConnectionPool:
    def __init__(self, index_name: str):
        self.index_name = index_name
        self.index = pinecone.Index(
            index_name, host="https://test-3-unx28qm.svc.aped-4627-b74a.pinecone.io"
        )

    def get_index(self):
        return self.index


# Creating a connection pool for Pinecone
connection_pool = PineconeConnectionPool(index_name="test-3")


class tenantId(BaseModel):
    tenantId: str


# retriever = vectorstore.similarity_search(
#     query="Vitafy",
#     k=1,
#     filter={"tenantId": tenantId}
# )

# retriever = vectorstore.as_retriever(
#     search_kwargs={"k": 3},
#     filter={"tenantId": tenantId},
# )


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


template = """You are an expert LLM assistant specialized in answering questions based solely on the information provided in the uploaded documents (PDF, DOCX, or TXT formats). Use only the information from the documents to respond accurately and clearly to each question.

Guidelines:
1. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
2. If the answer is not found in the uploaded documents, state, "The answer is not specifically mentioned in the provided documents."
3. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
4. Maintain a professional and helpful tone thinking you are giving service to the customer for their documents.
5. Answer for normal conversation questions like "Hi", "Hey", "Hello", "How are you?", and many others with the answer: "Hello, How can I assist you?".
6. If the question is on "summarize" or "summarization", then summarize the documents to (1/4)th the size of the original documents.

Question: {question}

Context: {context}

Answer:
"""
custom_rag_template = PromptTemplate.from_template(template)
# vectorstore.similarity_search(
#     query, k=3  # our search query  # return 3 most relevant docs
# )
# retriever = vectorstore.similarity_search(
#     search_kwargs={"k": 3, "filter": {"tenantId": tenantId}}
# )
# My_rag_chain = (
#     RunnableParallel(
#         {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     )
#     | custom_rag_template
#     | llm
#     | StrOutputParser()
# )

## My chain : Retriever(Pinecone) | custom_rag_template(prompt) | llm | StrOutputParser()


# $$$$$$$$$$ DOCS $$$$$$$$$
@app.get("/api/artificial-intelligence")
async def redirect_root_to_docs():
    return RedirectResponse("/docs")


# class tenantId(BaseModel):
#     tenantId: str


# def process_and_upsert_documents(
#     ids, texts, metadata, tenantId, text_splitter, embeddings, index
# ):
#     vectors = []

#     # Split into chunks, generate vectors, and add metadata
#     for doc_id, text, meta in zip(ids, texts, metadata):
#         chunks = text_splitter.split_text(text)  # Split document into chunks
#         for i, chunk in enumerate(chunks):
#             chunk_id = f"{doc_id}#chunk{i+1}"  # Create a unique ID for each chunk
#             vector = embeddings.embed_documents([chunk])[
#                 0
#             ]  # Generate embeddings for each chunk

#             # Prepare metadata with tenantId and fileName
#             chunk_metadata = {
#                 "fileName": meta["fileName"],
#                 "tenantId": tenantId,
#             }

#             # Preparing the vector for upsert
#             vectors.append(
#                 {"id": chunk_id, "values": vector, "metadata": chunk_metadata}
#             )

#     # Upsert vectors into Pinecone
#     if vectors:
#         index.upsert(vectors)

#     # Structured list of documents with IDs and names for the response
#     documents = [
#         {
#             "embeddedId": doc_id,
#             "fileName": meta["fileName"],
#             "tenantId": tenantId,
#         }
#         for doc_id, meta in zip(ids, metadata)
#     ]

#     return documents


# @app.post("/api/artificial-intelligence/upload")
# async def upload_files(
#     tenantId: str = Form(...),
#     files: List[UploadFile] = File(...),
#     background_task: BackgroundTasks = BackgroundTasks(),
# ):
#     """Upload multiple PDF, DOCX, and TXT files"""

#     # Create a directory for the tenant if it doesn't exist
#     tenant_directory = os.path.join(base_directory, tenantId)
#     os.makedirs(tenant_directory, exist_ok=True)

#     # Allowed file extensions
#     allowed_extensions = {".pdf", ".docx", ".txt"}

#     # Tracking file names to check for duplicates
#     fileName = set()

#     # Saving each uploaded file
#     for file in files:
#         if file.filename in fileName:
#             raise HTTPException(
#                 status_code=400, detail=f"Duplicate file detected: {file.filename}"
#             )

#         fileName.add(file.filename)

#         # Check for file extension
#         _, extension = os.path.splitext(file.filename)
#         if extension.lower() not in allowed_extensions:
#             raise HTTPException(
#                 status_code=400,
#                 detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",
#             )

#         # Define the destination path for the uploaded file
#         destination = os.path.join(tenant_directory, file.filename)

#         # Save the uploaded file to the tenant's directory
#         with open(destination, "wb") as buffer:
#             shutil.copyfileobj(file.file, buffer)

#     # Load the documents from the tenant's directory
#     docs = load_docs(tenant_directory)

#     # Extract texts and metadata for each document
#     texts = [
#         doc.page_content for doc in docs
#     ]  # Assuming docs have a page_content attribute
#     metadata = [
#         {"fileName": os.path.basename(doc.metadata["source"]), "tenantId": tenantId}
#         for doc in docs
#     ]



  

#     # Generate unique IDs for each document
#     ids = [str(uuid4()) for _ in range(len(texts))]

#     # Call the function to process and upsert the documents
#     background_task.add_task(
#         process_and_upsert_documents,
#         ids,
#         texts,
#         metadata,
#         tenantId,
#         text_splitter,
#         embeddings,
#         index,
#     )

#     # Return JSON response with document details
#     return JSONResponse(
#         content={"message": "Files uploaded successfully."},
#         status_code=200,
#     )






############### UPLOAD###########






from uuid import uuid4
from fastapi import BackgroundTasks, UploadFile, File, HTTPException, Form
from fastapi.responses import JSONResponse
import os
import shutil
from typing import List, Dict


class tenantId(BaseModel):
    tenantId: str


def process_and_upsert_documents(
    ids, texts, metadata, tenantId, text_splitter, embeddings, index
):
    vectors = []

    # Split into chunks, generate vectors, and add metadata
    for doc_id, text, meta in zip(ids, texts, metadata):
        chunks = text_splitter.split_text(text)  # Split document into chunks
        for i, chunk in enumerate(chunks):
            chunk_id = f"{doc_id}#chunk{i+1}"  # Create a unique ID for each chunk
            vector = embeddings.embed_documents([chunk])[0]  # Generate embeddings for each chunk

            # Prepare metadata with tenantId, fileName, source, and text
            chunk_metadata = {
                "fileName": meta["fileName"],
                "source": meta.get("source", "unknown"),
                "tenantId": tenantId,
                "text": chunk  # Store the chunk text in metadata
            }

            # Preparing the vector for upsert
            vectors.append(
                {"id": chunk_id, "values": vector, "metadata": chunk_metadata}
            )

    # Upsert vectors into Pinecone
    if vectors:
        index.upsert(vectors)

    # Structured list of documents with IDs and names for the response
    documents = [
        {
            "embeddedId": doc_id,
            "fileName": meta["fileName"],
            "tenantId": tenantId,
        }
        for doc_id, meta in zip(ids, metadata)
    ]

    return documents


@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks(),
):
    """Upload multiple PDF, DOCX, and TXT files"""

    # Create a directory for the tenant if it doesn't exist
    tenant_directory = os.path.join(base_directory, tenantId)
    os.makedirs(tenant_directory, exist_ok=True)

    # Allowed file extensions
    allowed_extensions = {".pdf", ".docx", ".txt"}

    # Tracking file names to check for duplicates
    fileName = set()

    # Saving each uploaded file
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}"
            )

        fileName.add(file.filename)

        # Check for file extension
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",
            )

        # Define the destination path for the uploaded file
        destination = os.path.join(tenant_directory, file.filename)

        # Save the uploaded file to the tenant's directory
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

    # Load the documents from the tenant's directory
    docs = load_docs(tenant_directory)

    # Extract texts and metadata for each document
    texts = [doc.page_content for doc in docs]  # Assuming docs have a page_content attribute
    metadata = [
        {
            "fileName": os.path.basename(doc.metadata["source"]),
            "source": doc.metadata["source"],
            "tenantId": tenantId
        }
        for doc in docs
    ]

    # Generate unique IDs for each document
    ids = [str(uuid4()) for _ in range(len(texts))]

    # Call the function to process and upsert the documents
    background_task.add_task(
        process_and_upsert_documents,
        ids,
        texts,
        metadata,
        tenantId,
        text_splitter,
        embeddings,
        index,
    )

    # Return JSON response with document details
    return JSONResponse(
        content={"message": "Files uploaded successfully."},
        status_code=200,
    )










@app.get("/api/artificial-intelligence/tenant_files")
async def get_tenant_files(tenantId: str = Query(...)):
    """Retrieve a list of uploaded files for a specific tenant"""

    # Tenant's directory
    tenant_directory = os.path.join(base_directory, tenantId)

    #  if tenant's directory exists
    if not os.path.exists(tenant_directory):
        raise HTTPException(
            status_code=404, detail=f"No documents found for tenantId: {tenantId}"
        )

    # all the files in the tenant's directory
    uploaded_files = os.listdir(tenant_directory)

    if not uploaded_files:
        return {"message": f"No files uploaded for tenantId: {tenantId}"}

    return {"tenantId": tenantId, "uploaded_files": uploaded_files}





# def run_rag_chain(query, tenantId):
#     # Perform a similarity search to retrieve relevant documents
#     retrieved_docs = vectorstore.similarity_search(
#         query, k=3, search_kwargs={"filter": {"tenantId": tenantId}}
#     )
    
#     # Format the retrieved documents
#     formatted_docs = format_docs(retrieved_docs)
    
#     # Prepare the context and question for the RAG chain
#     context = formatted_docs
#     question = query
    
#     # Run the RAG chain
#     response = (
#         RunnableParallel(
#             {"context": context, "question": RunnablePassthrough()}
#         )
#         | custom_rag_template
#         | llm
#         | StrOutputParser()
#     )
    
#     return response



# @app.get("/api/artificial-intelligence/prompts")
# async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
#     try:
#         # Retrieving the tenant directory based on the tenantId
#         tenant_directory = os.path.join(base_directory, tenantId)

#         # Checking if the tenant directory exists
#         if not os.path.exists(tenant_directory):
#             raise HTTPException(
#                 status_code=404,
#                 detail=f"No documents available for tenant ID {tenantId}.",
#             )

#         # Setting up the retriever to filter documents based on tenantId
#         retriever = vectorstore.as_retriever(
#             search_kwargs={"k": 1},
#             filter={"tenantId": tenantId},
#         )

#         # Retrieve relevant documents based on the keyword
#         retrieved_docs = retriever.get_relevant_documents(keyword)

#         # Formatting the retrieved documents into context text
#         # context = "\n\n".join(doc.page_content for doc in retrieved_docs)

#         # Use the RAG (Retrieval-Augmented Generation) chain to extract answers
#         # answer = qa.invoke(keyword)
#         answer = qa.invoke({"query": keyword})

#         # Extract relevant documents and metadata from the retrieved response
#         source_documents = [
#             {
#                 # "fileName": doc.metadata.get("fileName", "unknown"),
#                 # "tenantId": doc.metadata.get("tenantId", "unknown"),
#                 # "embeddedId": doc.metadata.get("embeddedId", "unknown"),
#                 # "content": doc.page_content,
#                 "answer": answer,
#             }
#             for doc in retrieved_docs
#         ]

#     except HTTPException as http_exc:
#         # Handle the tenant-specific exception
#         return {"error": http_exc.detail}

#     except Exception as e:
#         # Catch other exceptions and return the error message
#         return {"error": str(e)}

#     # Returning the source documents with metadata, content, and AI-generated answers
#     # return {"source_documents": answer}
#     return {"answer": answer}




################ CHAT ###############


@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    try:
        print(tenantId, keyword)
        vectorStore = initializeVectorStore()
        #retriever = vectorStore.as_retriever()
        retriever = vectorStore.as_retriever(
    
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': tenantId}  
    
    },
            }
)
        newQa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            #retriever=vectorStore.as_retriever(),
            return_source_documents=True,
        )

        #answer = newQa.invoke({"query": keyword})
        answer = newQa({"query": keyword})


        return {"answer": answer}
    except Exception as e:
        return {"error": str(e)}






############## DELETE ##############

class DeleteRequest(BaseModel):
    prefix: str

@app.delete("/api/artificial-intelligence/delete", summary="Delete documents by prefix", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """
    Deletes all document IDs in the Pinecone index that start with the given prefix.

    Parameters:
    - **prefix**: The prefix string to filter and delete documents by.

    Returns:
    - A JSON response with a success message or an error if no IDs were found with the given prefix.
    """
    prefix = request.prefix
    # List all IDs with the given prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]

    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")

    # Delete the IDs from the index
    index.delete(ids=ids_to_delete)
    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}

In [ ]:

############################# Add Q&A #############################

############################# Add Q&A #############################

############################# Add Q&A #############################
# @app.post("/api/artificial-intelligence/Add Q&A")
# async def upload_text(
#     tenantId: str = Form(...),
#     input_text: str = Form(...),
#     background_task: BackgroundTasks = BackgroundTasks(),
# ):
#     """
#     Upload and process raw text input and store it in Pinecone.
#     """
#     embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
#     vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

#     chunk_size = 1000
#     chunk_overlap = 20
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

#     split_docs = []
#     chunk_ids = []

#     uploaded_documents_path = "do_not_delete_uploaded_documents.json"

#     if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)

#     with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)

#     try:
#         document_id = str(uuid.uuid4())

#         metadata = {
#             "tenantId": tenantId,
#             "fileName": f"text_input_{document_id}.txt",
#         }

#         document = Document(page_content=input_text, metadata=metadata)

#         curr_split_docs = text_splitter.split_documents([document])

#         curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

#         split_docs += curr_split_docs
#         chunk_ids += curr_chunk_ids

#         upload_documents[document_id] = metadata

#         vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

#         with open(uploaded_documents_path, "w") as f:
#             json.dump(upload_documents, f, indent=4)

#         return {"status": "success", "message": "Text processed and stored successfully."}

#     except Exception as e:
#         raise HTTPException(status_code=500, detail=f"Error processing the text: {str(e)}")

In [ ]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langserve import add_routes
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from typing import Any
import uuid
import pylibmagic
from fastapi import Form, BackgroundTasks
from fastapi import Query
import io
import pinecone
from PyPDF2 import PdfReader
import docx
import shutil
import uvicorn
from typing import List, Dict
import numpy as np
import os
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import ServerlessSpec
# from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
# from langchain.document_loaders import DirectoryLoader
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from tqdm.auto import tqdm
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse, JSONResponse
import pinecone
import glob
from fastapi import FastAPI, File, UploadFile, HTTPException, Form
from fastapi.responses import JSONResponse
from uuid import uuid4
import time
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, Request
from fastapi import FastAPI, BackgroundTasks, UploadFile, File, Form, HTTPException
import getpass
import os
import concurrent.futures
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from PyPDF2 import PdfReader  # For reading PDF files
import docx
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

class MyModel(BaseModel):
    message: BaseMessage


class Config:
    arbitrary_types_allowed = True


app = FastAPI(
    title="LangChain Server",
    version="o1",
    description="",
)
# Set all CORS enabled origins
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["Processing-Time"] = str(process_time)
    return response


load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
directory = os.getenv("directory")
base_directory = os.getenv("base_directory")


if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if pinecone_api_key:
    os.environ["PINECONE_API_KEY"] = pinecone_api_key

# Verify that the keys are loaded
# print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
# print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")


pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
import time

index_name = "test-3"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)




@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks(),
):
    """Upload multiple PDF, DOCX, and TXT files"""
    dir_name = str(uuid4())
    tenant_directory = os.path.join(base_directory, dir_name)
    os.makedirs(tenant_directory, exist_ok=True)

    # Allowed file extensions
    allowed_extensions = {".pdf", ".docx", ".txt"}

    # Tracking file names to check for duplicates
    fileName = set()

    # Saving each uploaded file
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}"
            )

        fileName.add(file.filename)

        # Check for file extension
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",
            )

        # Define the destination path for the uploaded file
        destination = os.path.join(tenant_directory, file.filename)

        print('creating doc ' + destination)
        # Save the uploaded file to the tenant's directory
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)



    # Load the documents from the tenant's directory
    docs = load_docs(tenant_directory, tenantId)
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
    )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

    chunk_size = 5000 
    chunk_overlap = 20

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])

         # Generate a unique document ID
        document_id = str(uuid4())

        # Create unique IDs for each chunk with the document ID as a prefix
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids

        upload_documents[document_id] =  {"fileName": doc.metadata['filename'], "id": document_id, "tenantId": tenantId}

    # Add documents to vector store with unique chunk IDs
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f)

    shutil.rmtree(tenant_directory)

   
    

def load_docs(directory, tenantId):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    
    for doc in docs:
        doc.metadata['tenantId'] = tenantId

        # Extract the filename from the 'source' path
        doc.metadata['filename'] = os.path.basename(doc.metadata['source'])
        print("ooooooooo" + doc.metadata['filename'])

    return docs


@app.get("/api/artificial-intelligence/tenant_files")
async def retrieve_files(tenantId: str = Query(...)):
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"


    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    tenantFiles = []
    for item in list(upload_documents.values()):
        if item.get('tenantId') == tenantId:
            tenantFiles.append(item)
            
    return {"data": tenantFiles}



def initializeVectorStore():
    embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",                             #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
    )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    #model="gpt-4o",
    model="gpt-3.5-turbo",
    temperature=0.1,
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    try:
        print(tenantId, keyword)
        vectorStore = initializeVectorStore()
        #retriever = vectorStore.as_retriever()
        retriever = vectorStore.as_retriever(
    
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': tenantId}  
    
    },
            }
)
        newQa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            #retriever=vectorStore.as_retriever(),
            return_source_documents=True,
        )

        #answer = newQa.invoke({"query": keyword})
        answer = newQa({"query": keyword})


        return {"answer": answer}
    except Exception as e:
        return {"error": str(e)}




class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/delete", summary="Delete documents by prefix", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """
    Deletes all document IDs in the Pinecone index that start with the given prefix.

    Parameters:
    - **prefix**: The prefix string to filter and delete documents by.

    Returns:
    - A JSON response with a success message or an error if no IDs were found with the given prefix.
    """
    prefix = request.prefix
    # List all IDs with the given prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    
    print(ids_to_delete)

    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")

    # Delete the IDs from the index
    index.delete(ids=ids_to_delete)
    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}





if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
def initializeVectorStore():
    embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",                            
    )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    #model="gpt-4o",
    model="gpt-3.5-turbo",
    temperature=0.2,
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    try:
        print(tenantId, keyword)
        vectorStore = initializeVectorStore()
        #retriever = vectorStore.as_retriever()
        retriever = vectorStore.as_retriever(
    
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': tenantId}  
    
    },
            }
)
        newQa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            #retriever=vectorStore.as_retriever(),
            return_source_documents=True,
        )

        answer = newQa({"query": keyword})

    #For JSON Output
    #     return {"answer": answer}
    # except Exception as e:
    #     return {"error": str(e)}
    
    #For Plain Text Output
        return PlainTextResponse(answer["result"]) 

    except Exception as e:
        return PlainTextResponse(
            f"Error: {str(e)}. Please contact the support team for assistance."
        )


## Final code

In [ ]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
#from langserve import add_routes
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from typing import Any
import uuid
import pylibmagic
from fastapi import Form, BackgroundTasks
from fastapi import Query
import io
import pinecone
from PyPDF2 import PdfReader
import docx
import shutil
import uvicorn
from typing import List, Dict
import numpy as np
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever
from fastapi.responses import PlainTextResponse
from pinecone import ServerlessSpec
# from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
# from langchain.document_loaders import DirectoryLoader
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from tqdm.auto import tqdm
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse, JSONResponse
import pinecone
import glob
from fastapi import FastAPI, File, UploadFile, HTTPException, Form
from fastapi.responses import JSONResponse
from uuid import uuid4
import time
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, Request
from fastapi import FastAPI, BackgroundTasks, UploadFile, File, Form, HTTPException
import getpass
import os
import concurrent.futures
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from PyPDF2 import PdfReader  # For reading PDF files
import docx
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

class MyModel(BaseModel):
    message: BaseMessage


class Config:
    arbitrary_types_allowed = True


app = FastAPI(
    title="LangChain Server",
    version="o1",
    description="",
)
# Set all CORS enabled origins
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["Processing-Time"] = str(process_time)
    return response


load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")




if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if pinecone_api_key:
    os.environ["PINECONE_API_KEY"] = pinecone_api_key
if pinecone_index_name:
    os.environ["PINECONE_INDEX_NAME"] = pinecone_index_name

# Verify that the keys are loaded
# print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
# print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")


pinecone_api_key = os.environ.get("PINECONE_API_KEY")
index_name = os.environ.get("PINECONE_INDEX_NAME")

pc = Pinecone(api_key=pinecone_api_key)
import time

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks(),
):
    """Upload multiple PDF, DOCX, and TXT files"""
    dir_name = str(uuid4())

    os.makedirs(dir_name, exist_ok=True)

    # Allowed file extensions
    allowed_extensions = {".pdf", ".docx", ".txt"}

    # Tracking file names to check for duplicates
    fileName = set()

    # Saving each uploaded file
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}"
            )

        fileName.add(file.filename)

        # Check for file extension
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",
            )

        # Define the destination path for the uploaded file
        destination = os.path.join(dir_name, file.filename)

        print('creating doc ' + destination)
        # Save the uploaded file to the tenant's directory
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

    # Load the documents from the tenant's directory
    docs = load_docs(dir_name, tenantId)
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
    )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

    chunk_size = 1000 
    chunk_overlap = 20

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])

         # Generate a unique document ID
        document_id = str(uuid4())

        # Create unique IDs for each chunk with the document ID as a prefix
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids

        upload_documents[document_id] =  {"fileName": doc.metadata['filename'], "id": document_id, "tenantId": tenantId}

    # Add documents to vector store with unique chunk IDs
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    shutil.rmtree(dir_name)

    return {"status": "success", "message": "Files uploaded successfully."}

def load_docs(directory, tenantId):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    
    for doc in docs:
        doc.metadata['tenantId'] = tenantId

        # Extract the filename from the 'source' path
        doc.metadata['filename'] = os.path.basename(doc.metadata['source'])
        print("ooooooooo" + doc.metadata['filename'])

    return docs

############################# Web-URL #############################

############################# Web-URL #############################

############################# Web-URL #############################
@app.post("/api/artificial-intelligence/links")
async def upload_web_urls(
    tenantId: str = Form(...),
    urls: List[str] = Form(...),
):
    """
    Upload and process web URLs, then store content in Pinecone.
    """
    all_docs = []  

    for url in urls:
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()  
            
            for doc in docs:
                doc.metadata["tenantId"] = tenantId
                doc.metadata["source"] = url
            all_docs.extend(docs)
        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue

    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URLs provided or failed to fetch content.")

    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)


    chunk_size = 1000
    chunk_overlap = 20
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])

        # Generate a unique document ID
        document_id = str(uuid.uuid4())

        # Create unique IDs for each chunk with the document ID as a prefix
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)

        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": tenantId}

    # Add documents to vector store with unique chunk IDs
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    return {"status": "success", "message": "URLs processed and stored successfully."}

############################# qa #############################

############################# qa #############################

############################# qa #############################
@app.post("/api/artificial-intelligence/qa")
async def upload_question_answer(
    tenantId: str = Form(...),
    question: List[str] = Form(...),
    answer: List[str] = Form(...),
    background_task: BackgroundTasks = BackgroundTasks(),
):
    """
    Upload and process a question and answer pair and store it in Pinecone.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

    chunk_size = 1000
    chunk_overlap = 20
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    try:
        document_id = str(uuid.uuid4())

        combined_text = f"Question: {question}\nAnswer: {answer}"

        #document_id = str(uuid.uuid4())

        metadata = {
            "tenantId": tenantId,
            "fileName": f"Q&A_input_{document_id}.txt",
            "id": document_id,
            "question": question[0],  
            "answer": answer[0], 

        }

        document = Document(page_content=combined_text, metadata=metadata)

        curr_split_docs = text_splitter.split_documents([document])


        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs += curr_split_docs
        chunk_ids += curr_chunk_ids

        upload_documents[document_id] = metadata
        

        # Add to Pinecone vector store
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)

        #return {"status": "success", "message": "Question and Answer processed and stored successfully."}
        return {
            "status": "success",
            "message": "Question and Answer processed  successfully.",
            "id": document_id,
            "question": question[0],  
            "answer": answer[0]       
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing the question and answer: {str(e)}")

############################# edit-qa #############################

############################# edit-qa #############################

############################# edit-qa #############################
@app.put("/api/artificial-intelligence/qa_edit")
async def edit_question_answer(
    tenantId: str = Form(...),
    embeddedId: str = Form(...),
    question: List[str] = Form(...),
    answer: List[str] = Form(...),
    background_task: BackgroundTasks = BackgroundTasks(),
):
    """
    Edit a question and answer pair by replacing the content in the same embeddedId.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

    chunk_size = 1000
    chunk_overlap = 20
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    if embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")

    try:
        # Delete existing documents with the prefix
        ids_to_delete = [id for id in index.list(prefix=embeddedId)]
        index.delete(ids=ids_to_delete)

        # Update the content
        combined_text = f"Question: {question}\nAnswer: {answer}"

        metadata = {
            "tenantId": tenantId,
            "fileName": f"Q&A_input_{embeddedId}.txt",
            "id": embeddedId,
            "question": question[0],
            "answer": answer[0],
        }

        document = Document(page_content=combined_text, metadata=metadata)

        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{embeddedId}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        # Add updated documents to Pinecone
        vectorstore.add_documents(documents=curr_split_docs, ids=curr_chunk_ids)

        # Update metadata file
        upload_documents[embeddedId] = metadata

        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)

        return {
            "status": "success",
            "message": "Question and Answer edited successfully.",
            "id": embeddedId,
            "question": question[0],
            "answer": answer[0],
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error editing the question and answer: {str(e)}")
############################# tenant_files #############################

############################# tenant_files #############################

############################# tenant_files #############################

@app.get("/api/artificial-intelligence/tenant_files")
async def retrieve_files(tenantId: str = Query(...)):
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    tenantFiles = []
    for item in list(upload_documents.values()):
        if item.get('tenantId') == tenantId:
            tenantFiles.append(item)

    return {"data": tenantFiles}

############################# prompts #############################

############################# prompts #############################

############################# prompts #############################

def initializeVectorStore():
    embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",                             #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
    )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    #model="gpt-4o",
    model="gpt-3.5-turbo",
    temperature=0.0,
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    try:
        print(tenantId, keyword)
        vectorStore = initializeVectorStore()
        #retriever = vectorStore.as_retriever()
        retriever = vectorStore.as_retriever(
    
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': tenantId}  
    
    },
            }
)
        newQa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            #retriever=vectorStore.as_retriever(),
            return_source_documents=False,
        )

        #answer = newQa.invoke({"query": keyword})
        answer = newQa({"query": keyword})


           #For JSON Output
    #     return {"answer": answer}
    # except Exception as e:
    #     return {"error": str(e)}
    
    #For Plain Text Output
        return PlainTextResponse(answer["result"]) 

    except Exception as e:
        return PlainTextResponse(
            f"Error: {str(e)}. Please contact the support team for assistance."
        )


####################################### Delete ##########################

####################################### Delete ##########################

####################################### Delete ##########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/delete", summary="Delete documents by prefix", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """
    Deletes all document IDs in the Pinecone index that start with the given prefix.

    Parameters:
    - **prefix**: The prefix string to filter and delete documents by.

    Returns:
    - A JSON response with a success message or an error if no IDs were found with the given prefix.
    """
    prefix = request.prefix
    # List all IDs with the given prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    
    print(ids_to_delete)

    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")

    # Delete the IDs from the index
    index.delete(ids=ids_to_delete)

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    if prefix in upload_documents:
        del upload_documents[prefix]
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}

####################################### Delete qa ##########################

####################################### Delete qa##########################

####################################### Delete qa##########################

class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/qa", summary="Delete QA by prefix", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """
    Deletes all QA IDs in the Pinecone index that start with the given prefix.

    Parameters:
    - **prefix**: The prefix string to filter and delete QA by.

    Returns:
    - A JSON response with a success message or an error if no IDs were found with the given prefix.
    """
    prefix = request.prefix
    # List all IDs with the given prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    
    print(ids_to_delete)

    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No QA found with the given prefix.")

    # Delete the IDs from the index
    index.delete(ids=ids_to_delete)

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    if prefix in upload_documents:
        del upload_documents[prefix]
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    return {"message": f"Deleted {len(ids_to_delete)} QA with prefix '{prefix}'."}



if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
#from langserve import add_routes
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from typing import Any
import uuid
import pylibmagic
from fastapi import Query
import io
import docx
import shutil
import uvicorn
from typing import List, Dict
import numpy as np
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever
from fastapi.responses import PlainTextResponse
from pinecone import Pinecone, ServerlessSpec
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from tqdm.auto import tqdm
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse
import glob
from fastapi import  UploadFile
from fastapi.responses import JSONResponse
from uuid import uuid4
import time
from fastapi.middleware.cors import CORSMiddleware
from fastapi import  Request
from fastapi import  BackgroundTasks, UploadFile, Form
import getpass
import os
import concurrent.futures
from fastapi import  UploadFile
from PyPDF2 import PdfReader  
import docx
import json

class MyModel(BaseModel):
    message: BaseMessage
class Config:
    arbitrary_types_allowed = True

app = FastAPI(
    title="LangChain Server",
    version="version:vac0.1",
    description="")

origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"])

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["Processing-Time"] = str(process_time)
    return response


######################### keys #############################
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")
if not openai_api_key:
    raise EnvironmentError("OPENAI_API_KEY is not set. Please set it in your environment.")
if not pinecone_api_key:
    raise EnvironmentError("PINECONE_API_KEY is not set. Please set it in your environment.")
if not pinecone_index_name:
    raise EnvironmentError("PINECONE_INDEX_NAME is not set. Please set it in your environment.")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if pinecone_api_key:
    os.environ["PINECONE_API_KEY"] = pinecone_api_key
if pinecone_index_name:
    os.environ["PINECONE_INDEX_NAME"] = pinecone_index_name
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")


############### vectordatabase initializations ####################
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_index_name = os.environ.get("PINECONE_INDEX_NAME")
pc = Pinecone(api_key=pinecone_api_key)
pinecone_client = Pinecone(api_key=pinecone_api_key)
def initialize_pinecone_index(client: Pinecone, index_name: str):
    existing_indexes = [index_info["name"] for index_info in client.list_indexes()]
    if index_name not in existing_indexes:
        client.create_index(
            name=index_name,
            dimension=3072,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"))
        while not client.describe_index(index_name).status["ready"]:
            time.sleep(1)
    return client.Index(index_name)
index = initialize_pinecone_index(pinecone_client, pinecone_index_name)


############################# upload #############################
@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Upload and process multiple document files
        PDF, DOCX, and TXT files
    Args:
        tenantId: Identifier for the tenant
        files: List of files to upload
        background_task: Background task handler
    Returns:
        JSON response indicating upload status
    Raises:
        HTTPException: For invalid files or duplicates
    """
    dir_name = str(uuid4())
    os.makedirs(dir_name, exist_ok=True)
    allowed_extensions = {".pdf", ".docx", ".txt"}
    fileName = set()
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}")
        fileName.add(file.filename)
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",)
        destination = os.path.join(dir_name, file.filename)
        print('creating doc ' + destination)
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)
    docs = load_docs(dir_name, tenantId)
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    chunk_size = 500
    chunk_overlap = 50
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])
        document_id = str(uuid4())
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids
        upload_documents[document_id] =  {"fileName": doc.metadata['filename'], "id": document_id, "tenantId": tenantId}
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    shutil.rmtree(dir_name)
    return {"status": "success", "message": "Files uploaded successfully."}
def load_docs(directory, tenantId):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    for doc in docs:
        doc.metadata['tenantId'] = tenantId
        doc.metadata['filename'] = os.path.basename(doc.metadata['source'])
        print("ooooooooo" + doc.metadata['filename'])
    return docs


############################# links #############################
@app.post("/api/artificial-intelligence/links")
async def upload_web_urls(
    tenantId: str = Form(...),
    urls: List[str] = Form(...)):
    """
    Process and store web URL content
    Args:
        tenantId: Identifier for the tenant
        urls: List of URLs to process   
    Returns:
        JSON response indicating processing status   
    Raises:
        HTTPException: When no valid URLs are provided
    """
    all_docs = []  
    for url in urls:
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()  
            for doc in docs:
                doc.metadata["tenantId"] = tenantId
                doc.metadata["source"] = url
            all_docs.extend(docs)
        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue
    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URLs provided or failed to fetch content.")
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 500
    chunk_overlap = 50
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])
        document_id = str(uuid.uuid4())
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)
        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": tenantId}
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"status": "success", "message": "URLs processed and stored successfully."}


############################# qa@upload #############################
class QARequest(BaseModel):
    tenantId: str
    question: str
    answer: str
@app.post("/api/artificial-intelligence/qa")
async def upload_question_answer(
    request: QARequest,
    background_task: BackgroundTasks):
    """
    Upload and store a question-answer pair
    Args:
        request: QA request containing question and answer
        background_task: Background task handler    
    Returns:
        JSON response with QA details and status  
    Raises:
        HTTPException: For processing errors
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 500
    chunk_overlap = 50
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    try:
        document_id = str(uuid.uuid4())
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            "tenantId": request.tenantId,
            "fileName": f"Q&A_input_{document_id}.txt",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs += curr_split_docs
        chunk_ids += curr_chunk_ids
        upload_documents[document_id] = metadata
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer processed successfully.",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing the question and answer: {str(e)}")
    

############################# qa@edit #############################
class QAEditRequest(BaseModel):
    #tenantId: str
    embeddedId: str
    question: str
    answer: str
@app.put("/api/artificial-intelligence/qa")
async def edit_question_answer(
    request: QAEditRequest,  
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Edit a question and answer pair by replacing the content in the same embeddedId.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 500
    chunk_overlap = 50
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if request.embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    try:
        ids_to_delete = [id for id in index.list(prefix=request.embeddedId)]
        index.delete(ids=ids_to_delete)
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            #"tenantId": request.tenantId,
            "fileName": f"Q&A_input_{request.embeddedId}.txt",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{request.embeddedId}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        vectorstore.add_documents(documents=curr_split_docs, ids=curr_chunk_ids)
        upload_documents[request.embeddedId] = metadata
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer edited successfully.",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error editing the question and answer: {str(e)}")


####################################### qa@delete ##########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/qa", summary="Delete Question Answer", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete Question Answer matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No QA found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} QA with prefix '{prefix}'."}


############################# tenant_files #############################
# @app.get("/api/artificial-intelligence/tenant_files")
# async def retrieve_files(tenantId: str = Query(...)):
#     uploaded_documents_path = "do_not_delete_uploaded_documents.json"
#     if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)
#     with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)
#     tenantFiles = []
#     for item in list(upload_documents.values()):
#         if item.get('tenantId') == tenantId:
#             tenantFiles.append(item)
#     return {"data": tenantFiles}




@app.get("/api/artificial-intelligence/tenant_files")
async def retrieve_files(tenantId: str = Query(...)):
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    tenantFiles = []
    for item in list(upload_documents.values()):
        # Only include items that have either 'fileName' (from /upload) 
        # or 'source' (from /links) but not from /qa
        if (item.get('tenantId') == tenantId and 
            ('fileName' in item or 'source' in item) and 
            not item.get('fileName', '').startswith('Q&A_input_')):
            tenantFiles.append(item)

    return {"data": tenantFiles}


############################# prompts #############################
def initializeVectorStore():
    """Initialize and configure vector store with embeddings
    Returns:
        Configured PineconeVectorStore instance
    """
    embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    return vectorstore
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    """Process keyword prompts and retrieve answers
    Args:
        tenantId: Identifier for the tenant
        keyword: Search keyword/prompt  
    Returns:
        PlainText response with answer or error message
    """
    try:
        print(tenantId, keyword)
        vectorStore = initializeVectorStore()
        retriever = vectorStore.as_retriever(
        search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': tenantId} }} )
        newQa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=False)
        answer = newQa({"query": keyword})
        return PlainTextResponse(answer["result"]) 
    except Exception as e:
        return PlainTextResponse(
            f"Error: {str(e)}. Please contact the support team for assistance.")

####################################### delete ##########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/delete", summary="Delete Documents", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete documents matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}

















if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
 I am a specialized QA chatbot designed to provide accurate responses based solely on the given context. 
        Please input your question, and I will respond with the corresponding answer from the provided context. 
        I will not utilize any general knowledge or external information, ensuring focused and relevant answers.

In [ ]:
        If the user greets you, respond with the "Hello! How can I assist you today?"


In [5]:
pip install apify-client langchain-openai langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.docstore.document import Document
from langchain_community.utilities import ApifyWrapper
from langchain_community.utilities import GoogleSearchAPIWrapper


apify = ApifyWrapper()
# Call the Actor to obtain text from the crawled webpages
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "/docs/integrations/chat/"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

# Create a vector store based on the crawled data
index = VectorstoreIndexCreator().from_loaders([loader])

# Query the vector store
query = "Are any OpenAI chat models integrated in LangChain?"
result = index.query(query)
print(result)

ValidationError: 1 validation error for ApifyWrapper
  Value error, Did not find apify_api_token, please add an environment variable `APIFY_API_TOKEN` which contains it, or pass `apify_api_token` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [ ]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
#from langserve import add_routes
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from langchain.chains import LLMChain
from typing import Any
import uuid
import pylibmagic
from fastapi import Query
import io
import docx
import shutil
import uvicorn
from typing import List, Dict
import numpy as np
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever
from fastapi.responses import PlainTextResponse
from pinecone import Pinecone, ServerlessSpec
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from tqdm.auto import tqdm
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse
import glob
from fastapi import  UploadFile
from fastapi.responses import JSONResponse
from uuid import uuid4
import time
from fastapi.middleware.cors import CORSMiddleware
from fastapi import  Request
from fastapi import  BackgroundTasks, UploadFile, Form
import getpass
import os
import concurrent.futures
from fastapi import  UploadFile
from PyPDF2 import PdfReader  
import docx
import json

class MyModel(BaseModel):
    message: BaseMessage
class Config:
    arbitrary_types_allowed = True

app = FastAPI(
    title="LangChain Server",
    version="version:vac0.1",
    description="")

origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"])

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["Processing-Time"] = str(process_time)
    return response

######################### keys #############################
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")
if not openai_api_key:
    raise EnvironmentError("OPENAI_API_KEY is not set. Please set it in your environment.")
if not pinecone_api_key:
    raise EnvironmentError("PINECONE_API_KEY is not set. Please set it in your environment.")
if not pinecone_index_name:
    raise EnvironmentError("PINECONE_INDEX_NAME is not set. Please set it in your environment.")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if pinecone_api_key:
    os.environ["PINECONE_API_KEY"] = pinecone_api_key
if pinecone_index_name:
    os.environ["PINECONE_INDEX_NAME"] = pinecone_index_name
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

############### vectordatabase initializations ####################
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_index_name = os.environ.get("PINECONE_INDEX_NAME")
pc = Pinecone(api_key=pinecone_api_key)
pinecone_client = Pinecone(api_key=pinecone_api_key)
def initialize_pinecone_index(client: Pinecone, index_name: str):
    existing_indexes = [index_info["name"] for index_info in client.list_indexes()]
    if index_name not in existing_indexes:
        client.create_index(
            name=index_name,
            dimension=3072,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"))
        while not client.describe_index(index_name).status["ready"]:
            time.sleep(1)
    return client.Index(index_name)
index = initialize_pinecone_index(pinecone_client, pinecone_index_name)

############################# upload #############################
@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Upload and process multiple document files
        PDF, DOCX, and TXT files
    Args:
        tenantId: Identifier for the tenant
        files: List of files to upload
        background_task: Background task handler
    Returns:
        JSON response indicating upload status
    Raises:
        HTTPException: For invalid files or duplicates
    """
    dir_name = str(uuid4())
    os.makedirs(dir_name, exist_ok=True)
    allowed_extensions = {".pdf", ".docx", ".txt"}
    fileName = set()
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}")
        fileName.add(file.filename)
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",)
        destination = os.path.join(dir_name, file.filename)
        print('creating doc ' + destination)
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)
    docs = load_docs(dir_name, tenantId)
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    chunk_size = 800
    chunk_overlap = 150
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])
        document_id = str(uuid4())
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids
        upload_documents[document_id] =  {"fileName": doc.metadata['filename'], "id": document_id, "tenantId": tenantId}
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    shutil.rmtree(dir_name)
    return {"status": "success", "message": "Files uploaded successfully."}
def load_docs(directory, tenantId):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    for doc in docs:
        doc.metadata['tenantId'] = tenantId
        doc.metadata['filename'] = os.path.basename(doc.metadata['source'])
        print("ooooooooo" + doc.metadata['filename'])
    return docs

############################# links@upload #############################
class UploadLinkRequest(BaseModel):
    tenantId: str
    url: str 
@app.post("/api/artificial-intelligence/links")
async def upload_web_url(request: UploadLinkRequest, background_tasks: BackgroundTasks):

    """
    Process and store web URL content.
    
    Args:
        request: UploadLinkRequest containing tenantId and url.
    
    Returns:
        JSON response indicating processing status.
    
    Raises:
        HTTPException: When no valid url are provided.
    """
    all_docs = []  
    url = request.url.split(",")  
    for url in url:
        try:
            loader = WebBaseLoader(url.strip())  
            docs = loader.load()  
            for doc in docs:
                doc.metadata["tenantId"] = request.tenantId
                doc.metadata["source"] = url.strip()
            all_docs.extend(docs)
        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue
    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid url provided or failed to fetch content.")
    
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 300
    chunk_overlap = 50
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    
    document_id =  str(uuid.uuid4())
    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])
        document_id = document_id
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)
        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
    
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    
    return {"status": "success", "message": "url processed and stored successfully.", "data": {"documentId": document_id}}
############################# links@edit #############################
class UpdateLinkRequest(BaseModel):
    tenantId: str
    embeddedId: str
    url: str 
@app.put("/api/artificial-intelligence/links")
async def update_web_url(request: UpdateLinkRequest):
    """
    Update web URL content for a specific embeddedId.
    
    Args:
        request: UpdateLinkRequest containing tenantId, embeddedId, and url.
    
    Returns:
        JSON response indicating processing status.
    
    Raises:
        HTTPException: When no valid url are provided or if the embeddedId does not exist.
    """
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    
    if request.embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    
    try:
        all_docs = []
        url = request.url.split(",")  
        for url in url:
            try:
                loader = WebBaseLoader(url.strip())  
                docs = loader.load()  
                for doc in docs:
                    doc.metadata["tenantId"] = request.tenantId
                    doc.metadata["source"] = url.strip()
                all_docs.extend(docs)
            except Exception as e:
                print(f"Error loading URL {url}: {e}")
                continue
        
        if not all_docs:
            raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
        
        embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
        vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
        chunk_size = 3500
        chunk_overlap = 1000
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        split_docs = []
        chunk_ids = []
        
        existing_chunk_ids = [key for key in upload_documents if upload_documents[key]['id'] == request.embeddedId]
        
        for chunk_id in existing_chunk_ids:
            del upload_documents[chunk_id]  # Removing old chunks
            
        ids_to_delete = [id for id in index.list(prefix=request.embeddedId)]
        index.delete(ids=ids_to_delete)

        for doc in all_docs:
            curr_split_docs = text_splitter.split_documents([doc])
            document_id = request.embeddedId  # same embeddedId
            curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
            split_docs.extend(curr_split_docs)
            chunk_ids.extend(curr_chunk_ids)
            upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
        
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        
      

        return {
            "status": "success",
            "message": "URL updated successfully.",
            "embeddedId": request.embeddedId,
            "updated_url": request.url
        }
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error updating url: {str(e)}")
############################# links@delete #############################
class DeleteRequest(BaseModel):
    embeddedId: str
@app.delete("/api/artificial-intelligence/links", summary="Delete Documents", description="Delete all url documents in the Pinecone index that match the given prefix.")
async def delete_url(request: DeleteRequest):
    """Delete url documents matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching url documents found
    """
    prefix = request.embeddedId
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No url documents found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} documents with embeddedId '{prefix}'."}      
############################# qa@upload #############################
class QARequest(BaseModel):
    tenantId: str
    question: str
    answer: str
@app.post("/api/artificial-intelligence/qa")
async def upload_question_answer(
    request: QARequest,
    background_task: BackgroundTasks):
    """
    Upload and store a question-answer pair
    Args:
        request: QA request containing question and answer
        background_task: Background task handler    
    Returns:
        JSON response with QA details and status  
    Raises:
        HTTPException: For processing errors
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 20
    chunk_overlap = 5
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    try:
        document_id = str(uuid.uuid4())
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            "tenantId": request.tenantId,
            "fileName": f"Q&A_input_{document_id}.txt",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs += curr_split_docs
        chunk_ids += curr_chunk_ids
        upload_documents[document_id] = metadata
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer processed successfully.",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing the question and answer: {str(e)}")
    
############################# qa@edit #############################
class QAEditRequest(BaseModel):
    tenantId: str
    embeddedId: str
    question: str
    answer: str
@app.put("/api/artificial-intelligence/qa")
async def edit_question_answer(
    request: QAEditRequest,  
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Edit a question and answer pair by replacing the content in the same embeddedId.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 20
    chunk_overlap = 5
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if request.embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    try:
        ids_to_delete = [id for id in index.list(prefix=request.embeddedId)]
        index.delete(ids=ids_to_delete)
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            "tenantId": request.tenantId,
            "fileName": f"Q&A_input_{request.embeddedId}.txt",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{request.embeddedId}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        vectorstore.add_documents(documents=curr_split_docs, ids=curr_chunk_ids)
        upload_documents[request.embeddedId] = metadata
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer edited successfully.",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error editing the question and answer: {str(e)}")

####################################### qa@delete ##########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/qa", summary="Delete Question Answer", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete Question Answer matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No QA found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} QA with prefix '{prefix}'."}


############################# tenant_files #############################
# @app.get("/api/artificial-intelligence/tenant_files")
# async def retrieve_files(tenantId: str = Query(...)):
#     uploaded_documents_path = "do_not_delete_uploaded_documents.json"
#     if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)
#     with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)
#     tenantFiles = []
#     for item in list(upload_documents.values()):
#         if item.get('tenantId') == tenantId:
#             tenantFiles.append(item)
#     return {"data": tenantFiles}

@app.get("/api/artificial-intelligence/tenant_files")
async def retrieve_files(tenantId: str = Query(...)):
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    tenantFiles = []
    for item in list(upload_documents.values()):
        # Only include items that have either 'fileName' (from /upload) 
        # or 'source' (from /links) but not from /qa
        if (item.get('tenantId') == tenantId and 
            ('fileName' in item) and 
            not item.get('fileName', '').startswith('Q&A_input_')):
            tenantFiles.append(item)
    return {"data": tenantFiles}

############################# prompts #############################
def initializeVectorStore():
    """Initialize and configure vector store with embeddings
    Returns:
        Configured PineconeVectorStore instance
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    """Process keyword prompts and retrieve answers using improved RAG chain
    Args:
        tenantId: Identifier for the tenant
        keyword: Search keyword/prompt  
    Returns:
        PlainText response with answer or error message
    """
    try:
        print(tenantId, keyword)
        vectorstore = initializeVectorStore()
        template = """Answer the question based only on the following context:
        {context}
        User Question:
        {question}
         If the user greets you, respond with one of the following without searching information from knowledge base:
        - "Hello! How can I assist you today?"
        - "Hi there! What can I do for you?"
        - "Good day! How can I assist you?"
        - "Hey! What can I help you with?"
        - "Hi! How can I support you today?"
        - "Welcome! How may I help you?"
        - "Salutations! How can I assist you?"
        - "What's up? How can I help?"
        - "Good to see you! How can I assist?"

        Please provide a concise and accurate answer based on the context above. If the context does not contain  information to answer the question, respond with: "I don't have the information you're looking for, please provide additional details." And if it's greetings then show greetings response.
        Make sure your answer is relevant and accurate to the question and does not repeat the question itself..
        In response give only answer but not question.
        """
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={
                "k": 1,
                "filter": {
                    'tenantId': {'$eq': tenantId}
                }
            }
        )
        prompt = ChatPromptTemplate.from_template(template)
        rag_chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser())
        answer = rag_chain.invoke(keyword)
        return PlainTextResponse(answer)
    except Exception as e:
        error_msg = f"Error processing your request: {str(e)}. Please contact support for assistance."
        print(f"Error in prompts_keyword: {str(e)}") 
        return PlainTextResponse(error_msg, status_code=500)

############################ Summarizer #################################################
class SummarizationRequest(BaseModel):
    caseId: str
    assignedTo: str
    patientName: str
    onBehalfOf: str
    currentCaseStatus: str
    summarizationType: str
    notes: List[Any]

@app.post("/api/artificial-intelligence/summarize")
async def summarize(request: SummarizationRequest):
    """Combine JSON upload and summary generation into one endpoint."""
    try:
        json_data = {"caseId": request.caseId, "notes": request.notes, "patientName": request.patientName, "assignedTo": request.assignedTo, "currentCaseStatus": request.currentCaseStatus}
        noteLength = sum(len(note['note']) for note in request.notes)
        maxTokenToUse = 500
        
        if(noteLength <= 10):
            maxTokenToUse=200
        
        
        json_doc = Document(page_content=json.dumps(json_data), metadata={'fileName': 'query_json'})
        split_docs = [json_doc]
        print(f"Processing {len(split_docs)} documents.")
        
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.0,
            max_tokens=maxTokenToUse,
            top_p=1.0,
        )

        full_summary_prompt_template = f"""
        "{{split_docs}}"

        Please summarize the following notes directly to me, the {request.onBehalfOf} in a structured format, focusing on key actions and communications. Include the following sections:
        
        1. Initial Request: Briefly explain the initial issue or request.
        2. Key Updates: Highlight major actions taken, communications, and progress in chronological order, including dates.
        3. Current Status: Summarize the present situation or outcome of the case.

        Do not expose any ids

        Complete the sentence at the end with a full stop.
        Give html format
        """

        # unread_summary_prompt_template = f"""
        # "{{split_docs}}"

        # Please summarize the following case notes directly to me, the {request.onBehalfOf} in a paragraph format.
        
        # Do not expose any ids
        # Keep it simple and short.
        # Complete the sentence at the end with a full stop.
        # """


        unread_summary_prompt_template = f"""
            "{{split_docs}}"

            Please summarize the following case notes directly to me, the {request.onBehalfOf} in a paragraph format.
            
            Do not expose any ids
            Keep it simple and short.
            Complete the sentence at the end with a full stop.
            """


        prompt=""
        if(request.summarizationType == 'full'):
            prompt = full_summary_prompt_template
        else:
            prompt = unread_summary_prompt_template
        
        print(prompt)

        prompt = PromptTemplate.from_template(prompt)
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="split_docs")
        summary = stuff_chain.run(split_docs)

        sentences = summary
        print(sentences)

        # if len(sentences) > 1:
        #     summary = '.'.join(sentences[:-1]).strip()  
        # else:
        #     summary = sentences[0].strip()  

        # if not summary.endswith('.'):
        #     summary += '.'
        
        summary = sentences.replace("\n", " ")
        summary = summary.replace("```html", " ")
        return JSONResponse(content={
            "summary": summary
        })
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error during summarizing the documents: {str(e)}")

####################################### delete #########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/delete", summary="Delete Documents", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete documents matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}


if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
#from langserve import add_routes
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from langchain.chains import LLMChain
from typing import Any
import uuid
import pylibmagic
from fastapi import Query
import io
import docx
import shutil
import uvicorn
from typing import List, Dict
import numpy as np
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever
from fastapi.responses import PlainTextResponse
from pinecone import Pinecone, ServerlessSpec
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from tqdm.auto import tqdm
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse
import glob
from fastapi import  UploadFile
from fastapi.responses import JSONResponse
from uuid import uuid4
import time
from fastapi.middleware.cors import CORSMiddleware
from fastapi import  Request
from fastapi import  BackgroundTasks, UploadFile, Form
import getpass
import os
import concurrent.futures
from fastapi import  UploadFile
from PyPDF2 import PdfReader  
import docx
import json
from crawl4ai import AsyncWebCrawler


class MyModel(BaseModel):
    message: BaseMessage
class Config:
    arbitrary_types_allowed = True

app = FastAPI(
    title="LangChain Server",
    version="version:vac0.1",
    description="")

origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"])

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["Processing-Time"] = str(process_time)
    return response

######################### keys #############################
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")
if not openai_api_key:
    raise EnvironmentError("OPENAI_API_KEY is not set. Please set it in your environment.")
if not pinecone_api_key:
    raise EnvironmentError("PINECONE_API_KEY is not set. Please set it in your environment.")
if not pinecone_index_name:
    raise EnvironmentError("PINECONE_INDEX_NAME is not set. Please set it in your environment.")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if pinecone_api_key:
    os.environ["PINECONE_API_KEY"] = pinecone_api_key
if pinecone_index_name:
    os.environ["PINECONE_INDEX_NAME"] = pinecone_index_name
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

############### vectordatabase initializations ####################
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_index_name = os.environ.get("PINECONE_INDEX_NAME")
pc = Pinecone(api_key=pinecone_api_key)
pinecone_client = Pinecone(api_key=pinecone_api_key)
def initialize_pinecone_index(client: Pinecone, index_name: str):
    existing_indexes = [index_info["name"] for index_info in client.list_indexes()]
    if index_name not in existing_indexes:
        client.create_index(
            name=index_name,
            dimension=3072,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"))
        while not client.describe_index(index_name).status["ready"]:
            time.sleep(1)
    return client.Index(index_name)
index = initialize_pinecone_index(pinecone_client, pinecone_index_name)

############################# upload #############################
@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Upload and process multiple document files
        PDF, DOCX, and TXT files
    Args:
        tenantId: Identifier for the tenant
        files: List of files to upload
        background_task: Background task handler
    Returns:
        JSON response indicating upload status
    Raises:
        HTTPException: For invalid files or duplicates
    """
    dir_name = str(uuid4())
    os.makedirs(dir_name, exist_ok=True)
    allowed_extensions = {".pdf", ".docx", ".txt"}
    fileName = set()
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}")
        fileName.add(file.filename)
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",)
        destination = os.path.join(dir_name, file.filename)
        print('creating doc ' + destination)
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)
    docs = load_docs(dir_name, tenantId)
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    chunk_size = 800
    chunk_overlap = 150
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])
        document_id = str(uuid4())
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids
        upload_documents[document_id] =  {"fileName": doc.metadata['filename'], "id": document_id, "tenantId": tenantId}
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    shutil.rmtree(dir_name)
    return {"status": "success", "message": "Files uploaded successfully."}
def load_docs(directory, tenantId):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    for doc in docs:
        doc.metadata['tenantId'] = tenantId
        doc.metadata['filename'] = os.path.basename(doc.metadata['source'])
        print("ooooooooo" + doc.metadata['filename'])
    return docs

############################# links@upload #############################
class UploadLinkRequest(BaseModel):
    tenantId: str
    url: str 

@app.post("/api/artificial-intelligence/links")
async def upload_web_url(request: UploadLinkRequest, background_tasks: BackgroundTasks):
    """
    Process and store web URL content.
    
    Args:
        request: UploadLinkRequest containing tenantId and url.
    
    Returns:
        JSON response indicating processing status.
    
    Raises:
        HTTPException: When no valid url are provided.
    """
    all_docs = []  
    urls = request.url.split(",")  

    for url in urls:
        url = url.strip()
        try:
            async with AsyncWebCrawler(verbose=True) as crawler:
                result = await crawler.arun(url=url)  # Running the crawler on the URL
                extracted_content = result.markdown  # Extracted content in markdown format
                doc = Document(page_content=extracted_content, metadata={"source": url, "tenantId": request.tenantId})
                all_docs.append(doc)

        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue

    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
    
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=os.getenv("PINECONE_INDEX_NAME"), embedding=embeddings)
    chunk_size = 3500
    chunk_overlap = 1000
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    
    document_id = str(uuid.uuid4())
    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)
        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
    
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    
    return {"status": "success", "message": "URL processed and stored successfully.", "data": {"documentId": document_id}}
############################# links@edit #############################
class UpdateLinkRequest(BaseModel):
    tenantId: str
    embeddedId: str
    url: str 
@app.put("/api/artificial-intelligence/links")
async def update_web_url(request: UpdateLinkRequest):
    """
    Update web URL content for a specific embeddedId.
    
    Args:
        request: UpdateLinkRequest containing tenantId, embeddedId, and url.
    
    Returns:
        JSON response indicating processing status.
    
    Raises:
        HTTPException: When no valid url are provided or if the embeddedId does not exist.
    """
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    
    if request.embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    
    try:
        all_docs = []  
        urls = request.url.split(",")  

        for url in urls:
            url = url.strip()
            try:
                async with AsyncWebCrawler(verbose=True) as crawler:
                    result = await crawler.arun(url=url)  # Running the crawler on the URL
                    extracted_content = result.markdown  # Extracted content in markdown format
                    doc = Document(page_content=extracted_content, metadata={"source": url, "tenantId": request.tenantId})
                    all_docs.append(doc)

            except Exception as e:
                print(f"Error loading URL {url}: {e}")
                continue

        if not all_docs:
            raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
        
        embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
        vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
        chunk_size = 3500
        chunk_overlap = 1000
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        split_docs = []
        chunk_ids = []
        
        existing_chunk_ids = [key for key in upload_documents if upload_documents[key]['id'] == request.embeddedId]
        
        for chunk_id in existing_chunk_ids:
            del upload_documents[chunk_id]  
            
        ids_to_delete = [id for id in index.list(prefix=request.embeddedId)]
        index.delete(ids=ids_to_delete)

        for doc in all_docs:
            curr_split_docs = text_splitter.split_documents([doc])
            document_id = request.embeddedId  # same embeddedId
            curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
            split_docs.extend(curr_split_docs)
            chunk_ids.extend(curr_chunk_ids)
            upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
        
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        
      

        return {
            "status": "success",
            "message": "URL updated successfully.",
            "embeddedId": request.embeddedId,
            "updated_url": request.url
        }
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error updating url: {str(e)}")
############################# links@delete #############################
class DeleteRequest(BaseModel):
    embeddedId: str
@app.delete("/api/artificial-intelligence/links", summary="Delete Documents", description="Delete all url documents in the Pinecone index that match the given prefix.")
async def delete_url(request: DeleteRequest):
    """Delete url documents matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching url documents found
    """
    prefix = request.embeddedId
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No url documents found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} documents with embeddedId '{prefix}'."}      
############################# qa@upload #############################
class QARequest(BaseModel):
    tenantId: str
    question: str
    answer: str
@app.post("/api/artificial-intelligence/qa")
async def upload_question_answer(
    request: QARequest,
    background_task: BackgroundTasks):
    """
    Upload and store a question-answer pair
    Args:
        request: QA request containing question and answer
        background_task: Background task handler    
    Returns:
        JSON response with QA details and status  
    Raises:
        HTTPException: For processing errors
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 20
    chunk_overlap = 5
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    try:
        document_id = str(uuid.uuid4())
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            "tenantId": request.tenantId,
            "fileName": f"Q&A_input_{document_id}.txt",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs += curr_split_docs
        chunk_ids += curr_chunk_ids
        upload_documents[document_id] = metadata
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer processed successfully.",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing the question and answer: {str(e)}")
    
############################# qa@edit #############################
class QAEditRequest(BaseModel):
    tenantId: str
    embeddedId: str
    question: str
    answer: str
@app.put("/api/artificial-intelligence/qa")
async def edit_question_answer(
    request: QAEditRequest,  
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Edit a question and answer pair by replacing the content in the same embeddedId.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 20
    chunk_overlap = 5
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if request.embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    try:
        ids_to_delete = [id for id in index.list(prefix=request.embeddedId)]
        index.delete(ids=ids_to_delete)
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            "tenantId": request.tenantId,
            "fileName": f"Q&A_input_{request.embeddedId}.txt",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{request.embeddedId}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        vectorstore.add_documents(documents=curr_split_docs, ids=curr_chunk_ids)
        upload_documents[request.embeddedId] = metadata
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer edited successfully.",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error editing the question and answer: {str(e)}")

####################################### qa@delete ##########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/qa", summary="Delete Question Answer", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete Question Answer matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No QA found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} QA with prefix '{prefix}'."}


############################# tenant_files #############################
# @app.get("/api/artificial-intelligence/tenant_files")
# async def retrieve_files(tenantId: str = Query(...)):
#     uploaded_documents_path = "do_not_delete_uploaded_documents.json"
#     if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)
#     with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)
#     tenantFiles = []
#     for item in list(upload_documents.values()):
#         if item.get('tenantId') == tenantId:
#             tenantFiles.append(item)
#     return {"data": tenantFiles}

@app.get("/api/artificial-intelligence/tenant_files")
async def retrieve_files(tenantId: str = Query(...)):
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    tenantFiles = []
    for item in list(upload_documents.values()):
        # Only include items that have either 'fileName' (from /upload) 
        # or 'source' (from /links) but not from /qa
        if (item.get('tenantId') == tenantId and 
            ('fileName' in item) and 
            not item.get('fileName', '').startswith('Q&A_input_')):
            tenantFiles.append(item)
    return {"data": tenantFiles}

############################# prompts #############################
def initializeVectorStore():
    """Initialize and configure vector store with embeddings
    Returns:
        Configured PineconeVectorStore instance
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    """Process keyword prompts and retrieve answers using improved RAG chain
    Args:
        tenantId: Identifier for the tenant
        keyword: Search keyword/prompt  
    Returns:
        PlainText response with answer or error message
    """
    try:
        print(tenantId, keyword)
        vectorstore = initializeVectorStore()
        template = """Answer the question based only on the following context:
        {context}
        User Question:
        {question}
         If the user greets you, respond with one of the following without searching information from knowledge base:
        - "Hello! How can I assist you today?"
        - "Hi there! What can I do for you?"
        - "Good day! How can I assist you?"
        - "Hey! What can I help you with?"
        - "Hi! How can I support you today?"
        - "Welcome! How may I help you?"
        - "Salutations! How can I assist you?"
        - "What's up? How can I help?"
        - "Good to see you! How can I assist?"

        Please provide a concise and accurate answer based on the context above. If the context does not contain  information to answer the question, respond with: "I don't have the information you're looking for, please provide additional details." And if it's greetings then show greetings response.
        Make sure your answer is relevant and accurate to the question and does not repeat the question itself..
        In response give only answer but not question.
        """
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={
                "k": 1,
                "filter": {
                    'tenantId': {'$eq': tenantId}
                }
            }
        )
        prompt = ChatPromptTemplate.from_template(template)
        rag_chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser())
        answer = rag_chain.invoke(keyword)
        return PlainTextResponse(answer)
    except Exception as e:
        error_msg = f"Error processing your request: {str(e)}. Please contact support for assistance."
        print(f"Error in prompts_keyword: {str(e)}") 
        return PlainTextResponse(error_msg, status_code=500)

############################ Summarizer #################################################
class SummarizationRequest(BaseModel):
    caseId: str
    assignedTo: str
    patientName: str
    onBehalfOf: str
    currentCaseStatus: str
    summarizationType: str
    notes: List[Any]

@app.post("/api/artificial-intelligence/summarize")
async def summarize(request: SummarizationRequest):
    """Combine JSON upload and summary generation into one endpoint."""
    try:
        json_data = {"caseId": request.caseId, "notes": request.notes, "patientName": request.patientName, "assignedTo": request.assignedTo, "currentCaseStatus": request.currentCaseStatus}
        noteLength = sum(len(note['note']) for note in request.notes)
        maxTokenToUse = 500
        
        if(noteLength <= 10):
            maxTokenToUse=200
        
        
        json_doc = Document(page_content=json.dumps(json_data), metadata={'fileName': 'query_json'})
        split_docs = [json_doc]
        print(f"Processing {len(split_docs)} documents.")
        
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.0,
            max_tokens=maxTokenToUse,
            top_p=1.0,
        )

        full_summary_prompt_template = f"""
        "{{split_docs}}"

        Please summarize the following notes directly to me, the {request.onBehalfOf} in a structured format, focusing on key actions and communications. Include the following sections:
        
        1. Initial Request: Briefly explain the initial issue or request.
        2. Key Updates: Highlight major actions taken, communications, and progress in chronological order, including dates.
        3. Current Status: Summarize the present situation or outcome of the case.

        Do not expose any ids

        Complete the sentence at the end with a full stop.
        Give html format
        """

        # unread_summary_prompt_template = f"""
        # "{{split_docs}}"

        # Please summarize the following case notes directly to me, the {request.onBehalfOf} in a paragraph format.
        
        # Do not expose any ids
        # Keep it simple and short.
        # Complete the sentence at the end with a full stop.
        # """


        unread_summary_prompt_template = f"""
            "{{split_docs}}"

            Please summarize the following case notes directly to me, the {request.onBehalfOf} in a paragraph format.
            
            Do not expose any ids
            Keep it simple and short.
            Complete the sentence at the end with a full stop.
            """


        prompt=""
        if(request.summarizationType == 'full'):
            prompt = full_summary_prompt_template
        else:
            prompt = unread_summary_prompt_template
        
        print(prompt)

        prompt = PromptTemplate.from_template(prompt)
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="split_docs")
        summary = stuff_chain.run(split_docs)

        sentences = summary
        print(sentences)

        # if len(sentences) > 1:
        #     summary = '.'.join(sentences[:-1]).strip()  
        # else:
        #     summary = sentences[0].strip()  

        # if not summary.endswith('.'):
        #     summary += '.'
        
        summary = sentences.replace("\n", " ")
        summary = summary.replace("```html", " ")
        return JSONResponse(content={
            "summary": summary
        })
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error during summarizing the documents: {str(e)}")

####################################### delete #########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/delete", summary="Delete Documents", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete documents matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}


if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
#from langserve import add_routes
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from langchain.chains import LLMChain
from typing import Any
import uuid
import pylibmagic
from fastapi import Query
import io
import docx
import shutil
import uvicorn
from typing import List, Dict
import numpy as np
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever
from fastapi.responses import PlainTextResponse
from pinecone import Pinecone, ServerlessSpec
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from tqdm.auto import tqdm
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse
import glob
from fastapi import  UploadFile
from fastapi.responses import JSONResponse
from uuid import uuid4
import time
from fastapi.middleware.cors import CORSMiddleware
from fastapi import  Request
from fastapi import  BackgroundTasks, UploadFile, Form
import getpass
import os
import concurrent.futures
from fastapi import  UploadFile
from PyPDF2 import PdfReader  
import docx
import json
from crawl4ai import AsyncWebCrawler

class MyModel(BaseModel):
    message: BaseMessage
class Config:
    arbitrary_types_allowed = True

app = FastAPI(
    title="LangChain Server",
    version="version:vac0.1",
    description="")

origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"])

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["Processing-Time"] = str(process_time)
    return response

######################### keys #############################
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
langchain_project = os.getenv("LANGCHAIN_PROJECT")
langchain_endpoint = os.getenv("LANGCHAIN_ENDPOINT")
if langchain_api_key:
    os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
if langchain_project:
    os.environ["LANGCHAIN_PROJECT"] = langchain_project   
if langchain_endpoint:
    os.environ["LANGCHAIN_ENDPOINT"] = langchain_endpoint  
if not openai_api_key:
    raise EnvironmentError("OPENAI_API_KEY is not set. Please set it in your environment.")
if not pinecone_api_key:
    raise EnvironmentError("PINECONE_API_KEY is not set. Please set it in your environment.")
if not pinecone_index_name:
    raise EnvironmentError("PINECONE_INDEX_NAME is not set. Please set it in your environment.")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if pinecone_api_key:
    os.environ["PINECONE_API_KEY"] = pinecone_api_key
if pinecone_index_name:
    os.environ["PINECONE_INDEX_NAME"] = pinecone_index_name
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

############### vectordatabase initializations ####################
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_index_name = os.environ.get("PINECONE_INDEX_NAME")
pc = Pinecone(api_key=pinecone_api_key)
pinecone_client = Pinecone(api_key=pinecone_api_key)
def initialize_pinecone_index(client: Pinecone, index_name: str):
    existing_indexes = [index_info["name"] for index_info in client.list_indexes()]
    if index_name not in existing_indexes:
        client.create_index(
            name=index_name,
            dimension=3072,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"))
        while not client.describe_index(index_name).status["ready"]:
            time.sleep(1)
    return client.Index(index_name)
index = initialize_pinecone_index(pinecone_client, pinecone_index_name)

############################# upload #############################
@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Upload and process multiple document files
        PDF, DOCX, and TXT files
    Args:
        tenantId: Identifier for the tenant
        files: List of files to upload
        background_task: Background task handler
    Returns:
        JSON response indicating upload status
    Raises:
        HTTPException: For invalid files or duplicates
    """
    dir_name = str(uuid4())
    os.makedirs(dir_name, exist_ok=True)
    allowed_extensions = {".pdf", ".docx", ".txt"}
    fileName = set()
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}")
        fileName.add(file.filename)
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",)
        destination = os.path.join(dir_name, file.filename)
        print('creating doc ' + destination)
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)
    docs = load_docs(dir_name, tenantId)
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    chunk_size = 800
    chunk_overlap = 150
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])
        document_id = str(uuid4())
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids
        upload_documents[document_id] =  {"fileName": doc.metadata['filename'], "id": document_id, "tenantId": tenantId}
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    shutil.rmtree(dir_name)
    return {"status": "success", "message": "Files uploaded successfully."}
def load_docs(directory, tenantId):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    for doc in docs:
        doc.metadata['tenantId'] = tenantId
        doc.metadata['filename'] = os.path.basename(doc.metadata['source'])
        print("ooooooooo" + doc.metadata['filename'])
    return docs

############################# links@upload #############################
class UploadLinkRequest(BaseModel):
    tenantId: str
    url: str 

@app.post("/api/artificial-intelligence/links")
async def upload_web_url(request: UploadLinkRequest, background_tasks: BackgroundTasks):
    """
    Process and store web URL content.
    
    Args:
        request: UploadLinkRequest containing tenantId and url.
    
    Returns:
        JSON response indicating processing status.
    
    Raises:
        HTTPException: When no valid url are provided.
    """
    all_docs = []  
    urls = request.url.split(",")  

    for url in urls:
        url = url.strip()
        try:
            async with AsyncWebCrawler(verbose=True) as crawler:
                result = await crawler.arun(url=url)  # Running the crawler on the URL
                extracted_content = result.markdown  # Extracted content in markdown format
                doc = Document(page_content=extracted_content, metadata={"source": url, "tenantId": request.tenantId})
                all_docs.append(doc)

        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue

    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
    
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=os.getenv("PINECONE_INDEX_NAME"), embedding=embeddings)
    chunk_size = 3500
    chunk_overlap = 1000
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    
    document_id = str(uuid.uuid4())
    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)
        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
    
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    
    return {"status": "success", "message": "URL processed and stored successfully.", "data": {"documentId": document_id}}
############################# links@edit #############################
class UpdateLinkRequest(BaseModel):
    tenantId: str
    embeddedId: str
    url: str 
@app.put("/api/artificial-intelligence/links")
async def update_web_url(request: UpdateLinkRequest):
    """
    Update web URL content for a specific embeddedId.
    Args:
        request: UpdateLinkRequest containing tenantId, embeddedId, and url.
    Returns:
        JSON response indicating processing status.
    Raises:
        HTTPException: When no valid url are provided or if the embeddedId does not exist.
    """
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if request.embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    try:
        all_docs = []  
        urls = request.url.split(",")  

        for url in urls:
            url = url.strip()
            try:
                async with AsyncWebCrawler(verbose=True) as crawler:
                    result = await crawler.arun(url=url)  # Running the crawler on the URL
                    extracted_content = result.markdown  # Extracted content in markdown format
                    doc = Document(page_content=extracted_content, metadata={"source": url, "tenantId": request.tenantId})
                    all_docs.append(doc)

            except Exception as e:
                print(f"Error loading URL {url}: {e}")
                continue

        if not all_docs:
            raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
        
        embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
        vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
        chunk_size = 3500
        chunk_overlap = 1000
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        split_docs = []
        chunk_ids = []
        
        existing_chunk_ids = [key for key in upload_documents if upload_documents[key]['id'] == request.embeddedId]
        
        for chunk_id in existing_chunk_ids:
            del upload_documents[chunk_id]  # Removing old chunks
            
        ids_to_delete = [id for id in index.list(prefix=request.embeddedId)]
        index.delete(ids=ids_to_delete)

        for doc in all_docs:
            curr_split_docs = text_splitter.split_documents([doc])
            document_id = request.embeddedId  # same embeddedId
            curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
            split_docs.extend(curr_split_docs)
            chunk_ids.extend(curr_chunk_ids)
            upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}

        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "URL updated successfully.",
            "embeddedId": request.embeddedId,
            "updated_url": request.url
        }   
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error updating url: {str(e)}")
############################# links@delete #############################
class DeleteRequest(BaseModel):
    embeddedId: str
@app.delete("/api/artificial-intelligence/links", summary="Delete Documents", description="Delete all url documents in the Pinecone index that match the given prefix.")
async def delete_url(request: DeleteRequest):
    """Delete url documents matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching url documents found
    """
    prefix = request.embeddedId
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No url documents found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} documents with embeddedId '{prefix}'."}      
############################# qa@upload #############################
class QARequest(BaseModel):
    tenantId: str
    question: str
    answer: str
@app.post("/api/artificial-intelligence/qa")
async def upload_question_answer(
    request: QARequest,
    background_task: BackgroundTasks):
    """
    Upload and store a question-answer pair
    Args:
        request: QA request containing question and answer
        background_task: Background task handler    
    Returns:
        JSON response with QA details and status  
    Raises:
        HTTPException: For processing errors
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 20
    chunk_overlap = 5
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    try:
        document_id = str(uuid.uuid4())
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            "tenantId": request.tenantId,
            "fileName": f"Q&A_input_{document_id}.txt",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs += curr_split_docs
        chunk_ids += curr_chunk_ids
        upload_documents[document_id] = metadata
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer processed successfully.",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing the question and answer: {str(e)}")
    
############################# qa@edit #############################
class QAEditRequest(BaseModel):
    tenantId: str
    embeddedId: str
    question: str
    answer: str
@app.put("/api/artificial-intelligence/qa")
async def edit_question_answer(
    request: QAEditRequest,  
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Edit a question and answer pair by replacing the content in the same embeddedId.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 20
    chunk_overlap = 5
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if request.embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    try:
        ids_to_delete = [id for id in index.list(prefix=request.embeddedId)]
        index.delete(ids=ids_to_delete)
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            "tenantId": request.tenantId,
            "fileName": f"Q&A_input_{request.embeddedId}.txt",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{request.embeddedId}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        vectorstore.add_documents(documents=curr_split_docs, ids=curr_chunk_ids)
        upload_documents[request.embeddedId] = metadata
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer edited successfully.",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error editing the question and answer: {str(e)}")

####################################### qa@delete ##########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/qa", summary="Delete Question Answer", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete Question Answer matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No QA found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} QA with prefix '{prefix}'."}


############################# tenant_files #############################
# @app.get("/api/artificial-intelligence/tenant_files")
# async def retrieve_files(tenantId: str = Query(...)):
#     uploaded_documents_path = "do_not_delete_uploaded_documents.json"
#     if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)
#     with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)
#     tenantFiles = []
#     for item in list(upload_documents.values()):
#         if item.get('tenantId') == tenantId:
#             tenantFiles.append(item)
#     return {"data": tenantFiles}

@app.get("/api/artificial-intelligence/tenant_files")
async def retrieve_files(tenantId: str = Query(...)):
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    tenantFiles = []
    for item in list(upload_documents.values()):
        # Only include items that have either 'fileName' (from /upload) 
        # or 'source' (from /links) but not from /qa
        if (item.get('tenantId') == tenantId and 
            ('fileName' in item) and 
            not item.get('fileName', '').startswith('Q&A_input_')):
            tenantFiles.append(item)
    return {"data": tenantFiles}

############################# prompts #############################
def initializeVectorStore():
    """Initialize and configure vector store with embeddings
    Returns:
        Configured PineconeVectorStore instance
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    """Process keyword prompts and retrieve answers using improved RAG chain
    Args:
        tenantId: Identifier for the tenant
        keyword: Search keyword/prompt  
    Returns:
        PlainText response with answer or error message
    """
    try:
        print(tenantId, keyword)
        vectorstore = initializeVectorStore()
        template = """Answer the question based only on the following context:
        {context}
        User Question:
        {question}
         If the user greets you, respond with one of the following without searching information from knowledge base:
        - "Hello! How can I assist you today?"
        - "Hi there! What can I do for you?"
        - "Good day! How can I assist you?"
        - "Hey! What can I help you with?"
        - "Hi! How can I support you today?"
        - "Welcome! How may I help you?"
        - "Salutations! How can I assist you?"
        - "What's up? How can I help?"
        - "Good to see you! How can I assist?"

        Please provide a concise and accurate answer based on the context above. If the context does not contain  information to answer the question, respond with: "I don't have the information you're looking for, please provide additional details." And if it's greetings then show greetings response.
        Make sure your answer is relevant and accurate to the question and does not repeat the question itself..
        In response give only answer but not question.
        """
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={
                "k": 1,
                "filter": {
                    'tenantId': {'$eq': tenantId}
                }
            }
        )
        prompt = ChatPromptTemplate.from_template(template)
        rag_chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser())
        answer = rag_chain.invoke(keyword)
        return PlainTextResponse(answer)
    except Exception as e:
        error_msg = f"Error processing your request: {str(e)}. Please contact support for assistance."
        print(f"Error in prompts_keyword: {str(e)}") 
        return PlainTextResponse(error_msg, status_code=500)

############################ Summarizer #################################################
class SummarizationRequest(BaseModel):
    caseId: str
    assignedTo: str
    patientName: str
    onBehalfOf: str
    currentCaseStatus: str
    summarizationType: str
    notes: List[Any]

@app.post("/api/artificial-intelligence/summarize")
async def summarize(request: SummarizationRequest):
    """Combine JSON upload and summary generation into one endpoint."""
    try:
        json_data = {"caseId": request.caseId, "notes": request.notes, "patientName": request.patientName, "assignedTo": request.assignedTo, "currentCaseStatus": request.currentCaseStatus}
        noteLength = sum(len(note['note']) for note in request.notes)
        maxTokenToUse = 500
        
        if(noteLength <= 10):
            maxTokenToUse=200
        
        
        json_doc = Document(page_content=json.dumps(json_data), metadata={'fileName': 'query_json'})
        split_docs = [json_doc]
        print(f"Processing {len(split_docs)} documents.")
        
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.0,
            max_tokens=maxTokenToUse,
            top_p=1.0,
        )

        full_summary_prompt_template = f"""
        "{{split_docs}}"

        Please summarize the following notes directly to me, the {request.onBehalfOf} in a structured format, focusing on key actions and communications. Include the following sections:
        
        1. Initial Request: Briefly explain the initial issue or request.
        2. Key Updates: Highlight major actions taken, communications, and progress in chronological order, including dates.
        3. Current Status: Summarize the present situation or outcome of the case.

        Do not expose any ids

        Complete the sentence at the end with a full stop.
        Give html format
        """

        unread_summary_prompt_template = f"""
        "{{split_docs}}"

        Please summarize the following case notes directly to me, the {request.onBehalfOf} in a paragraph format.
        
        Do not expose any ids
        Keep it simple and short.
        Complete the sentence at the end with a full stop.
        """


        prompt=""
        if(request.summarizationType == 'full'):
            prompt = full_summary_prompt_template
        else:
            prompt = unread_summary_prompt_template
        
        print(prompt)

        prompt = PromptTemplate.from_template(prompt)
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="split_docs")
        summary = stuff_chain.run(split_docs)

        sentences = summary
        print(sentences)

        # if len(sentences) > 1:
        #     summary = '.'.join(sentences[:-1]).strip()  
        # else:
        #     summary = sentences[0].strip()  

        # if not summary.endswith('.'):
        #     summary += '.'
        
        summary = sentences.replace("\n", " ")
        summary = summary.replace("```html", " ")
        return JSONResponse(content={
            "summary": summary
        })
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error during summarizing the documents: {str(e)}")

####################################### delete #########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/delete", summary="Delete Documents", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete documents matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}


if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
############################# links@upload #############################
class UploadLinkRequest(BaseModel):
    tenantId: str
    url: str 

@app.post("/api/artificial-intelligence/links")
async def upload_web_url(request: UploadLinkRequest, background_tasks: BackgroundTasks):
    """
    Process and store web URL content.
    
    Args:
        request: UploadLinkRequest containing tenantId and url.
    
    Returns:
        JSON response indicating processing status.
    
    Raises:
        HTTPException: When no valid url are provided.
    """
    all_docs = []  
    urls = request.url.split(",")  

    for url in urls:
        url = url.strip()
        try:
            async with AsyncWebCrawler(verbose=True) as crawler:
                result = await crawler.arun(url=url)  # Running the crawler on the URL
                extracted_content = result.markdown  # Extracted content in markdown format
                doc = Document(page_content=extracted_content, metadata={"source": url, "tenantId": request.tenantId})
                all_docs.append(doc)

        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue

    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
    
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=os.getenv("PINECONE_INDEX_NAME"), embedding=embeddings)
    chunk_size = 4500
    chunk_overlap = 1000
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    
    document_id = str(uuid.uuid4())
    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)
        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
    
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    
    return {"status": "success", "message": "URL processed and stored successfully.", "data": {"documentId": document_id}}

In [ ]:
        full_summary_prompt_template = f"""
        "{{split_docs}}"

        Please summarize the following notes directly to me, the {request.onBehalfOf} in a structured format, focusing on key actions and communications. Include the following sections:
        
        1. Initial Request: Briefly explain the initial issue or request.
        2. Key Updates: Highlight major actions taken, communications, and progress in chronological order, including dates.
        3. Current Status: Summarize the present situation or outcome of the case.

        Do not expose any ids

        Complete the sentence at the end with a full stop.
        Give html format
        Don't give farewell messages like 'End of summary' or 'Thank you for your attention to this matter' at the end of summary.
        """

        unread_summary_prompt_template = f"""
        "{{split_docs}}"

        Please summarize the following case notes directly to me, the {request.onBehalfOf} in a paragraph format.
        
        Do not expose any ids
        Keep it simple and short.
        Complete the sentence at the end with a full stop.
        Don't give farewell messages like 'End of summary' or 'Thank you for your attention to this matter' at the end of summary.

        """

In [ ]:
 template = """Answer the question based only on the following context:
        {context}
        User Question:
        {question}
         If the user greets you, respond with one of the following without searching information from knowledge base:
        - "Hello! How can I assist you today?"
        - "Hi there! What can I do for you?"
        - "Good day! How can I assist you?"
        - "Hey! What can I help you with?"
        - "Hi! How can I support you today?"
        - "Welcome! How may I help you?"
        - "Salutations! How can I assist you?"
        - "What's up? How can I help?"
        - "Good to see you! How can I assist?"

        Please provide a concise and accurate answer based on the context above. If the context does not contain  information to answer the question, respond with: "I don't have the information you're looking for, please provide additional details." And if it's greetings then give greetings response.
        Make sure your answer is relevant and accurate to the question and does not repeat the question itself..
        In response give only answer but not question.
        """

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
import asyncio
from typing import Optional, Dict, Any
from contextlib import asynccontextmanager
import json
import time
# Import necessary functions from webrover.py
#from .webrover import setup_browser_2, main_agent_graph
import webrover
from webrover import * 


from fastapi.middleware.cors import CORSMiddleware


app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],  # Your Next.js app URL
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)



# Global variable to store browser session
browser_session: Dict[str, Any] = {
    "playwright": None,
    "browser": None,
    "page": None
}

# Global queue for browser events
browser_events = asyncio.Queue()

class BrowserSetupRequest(BaseModel):
    url: str = "https://www.google.com"

class QueryRequest(BaseModel):
    query: str

@app.post("/setup-browser")
async def setup_browser(request: BrowserSetupRequest):
    try:
        # Clear any existing session
        if browser_session["playwright"]:
            await cleanup_browser()
            
        # Setup new browser session
        playwright, browser, page = await setup_browser_2(request.url)
        
        # Store session info
        browser_session.update({
            "playwright": playwright,
            "browser": browser,
            "page": page
        })
        
        return {"status": "success", "message": "Browser setup complete"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to setup browser: {str(e)}")

@app.post("/cleanup")
async def cleanup_browser():
    try:
        if browser_session["page"]:
            await browser_session["page"].close()
        if browser_session["browser"]:
            await browser_session["browser"].close()
        if browser_session["playwright"]:
            await browser_session["playwright"].stop()
            
        # Reset session
        browser_session.update({
            "playwright": None,
            "browser": None,
            "page": None
        })
        
        return {"status": "success", "message": "Browser cleanup complete"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to cleanup browser: {str(e)}")

async def emit_browser_event(event_type: str, data: Dict[str, Any]):
    await browser_events.put({
        "type": event_type,
        "data": data
    })

@app.get("/browser-events")
async def browser_events_endpoint():
    async def event_generator():
        while True:
            try:
                event = await browser_events.get()
                yield f"data: {json.dumps(event)}\n\n"
            except asyncio.CancelledError:
                break
    
    return StreamingResponse(
        event_generator(),
        media_type="text/event-stream",
        headers={
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
        }
    )

async def stream_agent_response(query: str, page):
    try:
        initial_state = {
            "input": query,
            "page": page,
            "image": "",
            "master_plan": None,
            "bboxes": [],
            "actions_taken": [],
            "action": None,
            "last_action": "",
            "notes": [],
            "answer": ""
        }
        
        # Keep track of last event for potential retries
        last_event = None
        retry_count = 0
        max_retries = 3
        
        async for event in main_agent_graph.astream(
            initial_state,
             {"recursion_limit": 400}
            ):
            try:
                # Send periodic keepalive to prevent timeout
                yield f"data: {{\n  \"type\": \"keepalive\",\n  \"timestamp\": {time.time()}\n}}\n\n"
                
                if isinstance(event, dict):
                    last_event = event
                    
                    if "parse_action" in event:
                        action = event["parse_action"]["action"]
                        thought = event["parse_action"]["notes"][-1]
                        
                        # Ensure proper encoding and escaping of JSON
                        thought_json = json.dumps(thought, ensure_ascii=False)
                        yield f"data: {{\n  \"type\": \"thought\",\n  \"content\": {thought_json}\n}}\n\n"
                        
                        if isinstance(action, dict):
                            action_json = json.dumps(action, ensure_ascii=False)
                            yield f"data: {{\n  \"type\": \"action\",\n  \"content\": {action_json}\n}}\n\n"
                            
                            # Handle browser events
                            action_type = action.get("action", "")
                            if action_type == "goto":
                                await emit_browser_event("navigation", {
                                    "url": action["args"],
                                    "status": "loading"
                                })
                    
                    if "answer_node" in event:
                        answer = event["answer_node"]["answer"]
                        answer_json = json.dumps(answer, ensure_ascii=False)
                        yield f"data: {{\n  \"type\": \"final_answer\",\n  \"content\": {answer_json}\n}}\n\n"
                        
                    # Reset retry count on successful event
                    retry_count = 0
                    
            except Exception as e:
                print(f"Error processing event: {str(e)}")
                retry_count += 1
                if retry_count <= max_retries and last_event:
                    # Retry last event
                    yield f"data: {{\n  \"type\": \"retry\",\n  \"content\": \"Retrying last action...\"\n}}\n\n"
                    continue
                else:
                    raise e
                    
    except Exception as e:
        error_json = json.dumps(str(e), ensure_ascii=False)
        yield f"data: {{\n  \"type\": \"error\",\n  \"content\": {error_json}\n}}\n\n"
    finally:
        # Ensure proper stream closure
        yield f"data: {{\n  \"type\": \"end\",\n  \"content\": \"Stream completed\"\n}}\n\n"

@app.post("/query")
async def query_agent(request: QueryRequest):
    if not browser_session["page"]:
        raise HTTPException(status_code=400, detail="Browser not initialized. Call /setup-browser first")
    
    return StreamingResponse(
        stream_agent_response(request.query, browser_session["page"]),
        media_type="text/event-stream"
    )

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=5000)

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
try:
    nltk.download('punkt', quiet=True)
except Exception as e:
    print(f"Warning: NLTK download failed, but continuing execution: {e}")

In [ ]:
############################# prompts #############################
def initializeVectorStore():
    """Initialize and configure vector store with embeddings
    Returns:
        Configured PineconeVectorStore instance
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    """Process keyword prompts and retrieve answers using improved RAG chain
    Args:
        tenantId: Identifier for the tenant
        keyword: Search keyword/prompt  
    Returns:
        PlainText response with answer or error message
    """
    try:
        print(tenantId, keyword)
        vectorstore = initializeVectorStore()
        template = """Answer the question based only on the following context:
        {context}
        User Question:
        {question}
         If the user greets you, respond with one of the following without searching information from knowledge base:
        - "Hello! How can I assist you today?"
        - "Hi there! What can I do for you?"
        - "Good day! How can I assist you?"
        - "Hey! What can I help you with?"
        - "Hi! How can I support you today?"
        - "Welcome! How may I help you?"
        - "Salutations! How can I assist you?"
        - "What's up? How can I help?"
        - "Good to see you! How can I assist?"

        Instructions for answering:
        1. Extract relevant information directly from the web content
        2. If the question is about specific data or facts, cite them accurately
        3. If the content contains multiple relevant sections, synthesize them into a coherent answer
        4. For technical or complex topics, provide clear, simplified explanations
        5. If the information is not available in the context, respond with: "This information is not available in the source content. Please try a different question or provide more details."
        6. For greetings, use one of the greeting responses above
        7. If the user asks questions that are synonyms or rephrased versions of questions in URL links, provide relevant answers based on the intent of the question, not just the exact wording.
        8. When answering, focus on matching the intent of the user's question, even if the exact words are not present in the context. Use contextual understanding to infer the meaning and provide accurate responses.
        9. If the question is ambiguous or unclear, ask for clarification to ensure the response aligns with the user's intent.
        10. For numerical data or specific metrics (e.g., measurements, statistics, or specifications), provide the exact match from the context.
        Remember:- For example: The secondary sensor of the Google Pixel 9 with 48 MP ultra-wide-angle, f2.2 and field of view (FOV) is 125.8˚and
                                 The secondary sensor of the Galaxy S25 with 12 MP ultra-wide-angle, f/2.2 aperture and field of view (FOV) is 120˚ 
        """

        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={
                "k": 5,  
                "filter": {
                    'tenantId': {'$eq': tenantId}
                }
            }
        )

        # Getting initial documents
        docs = retriever.get_relevant_documents(keyword)

        # Preparing documents for BM25 reranking
        corpus = [doc.page_content for doc in docs]
        tokenized_corpus = [word_tokenize(doc.lower()) for doc in corpus]
        bm25 = BM25Okapi(tokenized_corpus)

        #  BM25 scores for the keyword
        tokenized_query = word_tokenize(keyword.lower())
        bm25_scores = bm25.get_scores(tokenized_query)

        # Combining vector similarity with BM25 scores
        reranked_pairs = list(zip(docs, bm25_scores))
        reranked_pairs.sort(key=lambda x: x[1], reverse=True)

        # Taking top document after reranking
        reranked_docs = [pair[0] for pair in reranked_pairs[:1]]
        
        prompt = ChatPromptTemplate.from_template(template)
        rag_chain = (
            {"context": lambda x: reranked_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser())
        
        answer = rag_chain.invoke(keyword)
        return PlainTextResponse(answer)
        
    except Exception as e:
        error_msg = f"Error processing your request: {str(e)}. Please contact support for assistance."
        print(f"Error in prompts_keyword: {str(e)}") 
        return PlainTextResponse(error_msg, status_code=500)    

In [ ]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
#from langserve import add_routes
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from langchain.chains import LLMChain
from enum import Enum
from typing import Any
import uuid
import pylibmagic
from fastapi import Query
import random
import io
import docx
import shutil
import uvicorn
from typing import List, Dict
import numpy as np
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever
from fastapi.responses import PlainTextResponse
from pinecone import Pinecone, ServerlessSpec
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from tqdm.auto import tqdm
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse
import glob
from fastapi import  UploadFile
from fastapi.responses import JSONResponse
from uuid import uuid4
import time
from fastapi.middleware.cors import CORSMiddleware
from fastapi import  Request
from fastapi import  BackgroundTasks, UploadFile, Form
import getpass
import os
import concurrent.futures
from fastapi import  UploadFile
from PyPDF2 import PdfReader  
import docx
import json
from crawl4ai import AsyncWebCrawler
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

class MyModel(BaseModel):
    message: BaseMessage
class Config:
    arbitrary_types_allowed = True

app = FastAPI(
    title="LangChain Server",
    version="version:vac0.1",
    description="")

origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"])

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["Processing-Time"] = str(process_time)
    return response

######################### keys #############################
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")

if not openai_api_key:
    raise EnvironmentError("OPENAI_API_KEY is not set. Please set it in your environment.")
if not pinecone_api_key:
    raise EnvironmentError("PINECONE_API_KEY is not set. Please set it in your environment.")
if not pinecone_index_name:
    raise EnvironmentError("PINECONE_INDEX_NAME is not set. Please set it in your environment.")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if pinecone_api_key:
    os.environ["PINECONE_API_KEY"] = pinecone_api_key
if pinecone_index_name:
    os.environ["PINECONE_INDEX_NAME"] = pinecone_index_name
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

############### vectordatabase initializations ####################
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_index_name = os.environ.get("PINECONE_INDEX_NAME")
pc = Pinecone(api_key=pinecone_api_key)
pinecone_client = Pinecone(api_key=pinecone_api_key)
def initialize_pinecone_index(client: Pinecone, index_name: str):
    existing_indexes = [index_info["name"] for index_info in client.list_indexes()]
    if index_name not in existing_indexes:
        client.create_index(
            name=index_name,
            dimension=3072,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"))
        while not client.describe_index(index_name).status["ready"]:
            time.sleep(1)
    return client.Index(index_name)
index = initialize_pinecone_index(pinecone_client, pinecone_index_name)

############################# upload #############################
@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Upload and process multiple document files
        PDF, DOCX, and TXT files
    Args:
        tenantId: Identifier for the tenant
        files: List of files to upload
        background_task: Background task handler
    Returns:
        JSON response indicating upload status
    Raises:
        HTTPException: For invalid files or duplicates
    """
    dir_name = str(uuid4())
    os.makedirs(dir_name, exist_ok=True)
    allowed_extensions = {".pdf", ".docx", ".txt"}
    fileName = set()
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}")
        fileName.add(file.filename)
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",)
        destination = os.path.join(dir_name, file.filename)
        print('creating doc ' + destination)
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)
    docs = load_docs(dir_name, tenantId)
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    chunk_size = 800
    chunk_overlap = 150
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])
        document_id = str(uuid4())
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids
        upload_documents[document_id] =  {"fileName": doc.metadata['filename'], "id": document_id, "tenantId": tenantId}
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    shutil.rmtree(dir_name)
    return {"status": "success", "message": "Files uploaded successfully."}
def load_docs(directory, tenantId):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    for doc in docs:
        doc.metadata['tenantId'] = tenantId
        doc.metadata['filename'] = os.path.basename(doc.metadata['source'])
        print("ooooooooo" + doc.metadata['filename'])
    return docs

############################# links@upload #############################
class UploadLinkRequest(BaseModel):
    tenantId: str
    url: str 
    name: str

class DocumentType(str, Enum):
    FILE = "File"
    LINK = "Link"

@app.post("/api/artificial-intelligence/links")
async def upload_web_url(request: UploadLinkRequest, background_tasks: BackgroundTasks):
    """
    Process and store web URL content.
    
    Args:
        request: UploadLinkRequest containing tenantId and url.
    
    Returns:
        JSON response indicating processing status.
    
    Raises:
        HTTPException: When no valid url are provided.
    """
    all_docs = []  
    urls = request.url.split(",")  

    for url in urls:
        url = url.strip()
        try:
            async with AsyncWebCrawler(verbose=True) as crawler:
                result = await crawler.arun(url=url)  # Running the crawler on the URL
                extracted_content = result.markdown  # Extracted content in markdown format
                doc = Document(page_content=extracted_content, metadata={"source": url, "fileName":request.name, "tenantId": request.tenantId,"type": DocumentType.LINK })
                all_docs.append(doc)

        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue

    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
    
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=os.getenv("PINECONE_INDEX_NAME"), embedding=embeddings)
    chunk_size = 12500
    chunk_overlap = 1000
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    
    document_id = str(uuid.uuid4())
    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)
        upload_documents[document_id] = {"source": doc.metadata["source"],"fileName":request.name, "id": document_id, "tenantId": request.tenantId, "type": DocumentType.LINK}
    
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    
    return {"status": "success", "message": "URL processed and stored successfully.", "data": {"documentId": document_id}}
############################# links@edit #############################
class UpdateLinkRequest(BaseModel):
    tenantId: str
    embeddedId: str
    url: str 
    name: str

# class DocumentType(str, Enum):
#     FILE = "File"
#     LINK = "Link"    
@app.put("/api/artificial-intelligence/links")
async def update_web_url(request: UpdateLinkRequest):
    """
    Update web URL content for a specific embeddedId.
    Args:
        request: UpdateLinkRequest containing tenantId, embeddedId, and url.
    Returns:
        JSON response indicating processing status.
    Raises:
        HTTPException: When no valid url are provided or if the embeddedId does not exist.
    """
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if request.embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    try:
        all_docs = []  
        urls = request.url.split(",")  

        for url in urls:
            url = url.strip()
            try:
                async with AsyncWebCrawler(verbose=True) as crawler:
                    result = await crawler.arun(url=url)  # Running the crawler on the URL
                    extracted_content = result.markdown  # Extracted content in markdown format
                    doc = Document(page_content=extracted_content, metadata={"source": url,"fileName":request.name, "tenantId": request.tenantId, "type": DocumentType.LINK})
                    all_docs.append(doc)

            except Exception as e:
                print(f"Error loading URL {url}: {e}")
                continue

        if not all_docs:
            raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
        
        embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
        vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
        chunk_size = 6500
        chunk_overlap = 1000
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        split_docs = []
        chunk_ids = []
        
        existing_chunk_ids = [key for key in upload_documents if upload_documents[key]['id'] == request.embeddedId]
        
        for chunk_id in existing_chunk_ids:
            del upload_documents[chunk_id]  # Removing old chunks
            
        ids_to_delete = [id for id in index.list(prefix=request.embeddedId)]
        index.delete(ids=ids_to_delete)

        for doc in all_docs:
            curr_split_docs = text_splitter.split_documents([doc])
            document_id = request.embeddedId  # same embeddedId
            curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
            split_docs.extend(curr_split_docs)
            chunk_ids.extend(curr_chunk_ids)
            upload_documents[document_id] = {"source": doc.metadata["source"],"id": document_id, "fileName":request.name, "tenantId": request.tenantId, "type": DocumentType.LINK}

        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "URL updated successfully.",
            "embeddedId": request.embeddedId,
            "type": DocumentType.LINK,
            "updated_url": request.url
        }   
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error updating url: {str(e)}")
############################# links@delete #############################
class DeleteRequest(BaseModel):
    embeddedId: str
@app.delete("/api/artificial-intelligence/links", summary="Delete Documents", description="Delete all url documents in the Pinecone index that match the given prefix.")
async def delete_url(request: DeleteRequest):
    """Delete url documents matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching url documents found
    """
    prefix = request.embeddedId
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No url documents found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} documents with embeddedId '{prefix}'."}      
############################# qa@upload #############################
class QARequest(BaseModel):
    tenantId: str
    question: str
    answer: str
@app.post("/api/artificial-intelligence/qa")
async def upload_question_answer(
    request: QARequest,
    background_task: BackgroundTasks):
    """
    Upload and store a question-answer pair
    Args:
        request: QA request containing question and answer
        background_task: Background task handler    
    Returns:
        JSON response with QA details and status  
    Raises:
        HTTPException: For processing errors
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 500
    chunk_overlap = 100
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    try:
        document_id = str(uuid.uuid4())
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            "tenantId": request.tenantId,
            "fileName": f"Q&A_input_{document_id}.txt",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs += curr_split_docs
        chunk_ids += curr_chunk_ids
        upload_documents[document_id] = metadata
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer processed successfully.",
            "id": document_id,
            "question": request.question,  
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing the question and answer: {str(e)}")      
############################# qa@edit #############################
class QAEditRequest(BaseModel):
    tenantId: str
    embeddedId: str
    question: str
    answer: str
@app.put("/api/artificial-intelligence/qa")
async def edit_question_answer(
    request: QAEditRequest,  
    background_task: BackgroundTasks = BackgroundTasks()):
    """
    Edit a question and answer pair by replacing the content in the same embeddedId.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 20
    chunk_overlap = 5
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if request.embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    try:
        ids_to_delete = [id for id in index.list(prefix=request.embeddedId)]
        index.delete(ids=ids_to_delete)
        combined_text = f"Question: {request.question}\nAnswer: {request.answer}"
        metadata = {
            "tenantId": request.tenantId,
            "fileName": f"Q&A_input_{request.embeddedId}.txt",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer}
        document = Document(page_content=combined_text, metadata=metadata)
        curr_split_docs = text_splitter.split_documents([document])
        curr_chunk_ids = [f"{request.embeddedId}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        vectorstore.add_documents(documents=curr_split_docs, ids=curr_chunk_ids)
        upload_documents[request.embeddedId] = metadata
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        return {
            "status": "success",
            "message": "Question and Answer edited successfully.",
            "id": request.embeddedId,
            "question": request.question,
            "answer": request.answer }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error editing the question and answer: {str(e)}")

####################################### qa@delete ##########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/qa", summary="Delete Question Answer", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete Question Answer matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No QA found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} QA with prefix '{prefix}'."}


############################# tenant_files #############################
# @app.get("/api/artificial-intelligence/tenant_files")
# async def retrieve_files(tenantId: str = Query(...)):
#     uploaded_documents_path = "do_not_delete_uploaded_documents.json"
#     if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)
#     with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)
#     tenantFiles = []
#     for item in list(upload_documents.values()):
#         if item.get('tenantId') == tenantId:
#             tenantFiles.append(item)
#     return {"data": tenantFiles}

############################# tenant_files #############################
@app.get("/api/artificial-intelligence/tenant_files")
async def retrieve_files(tenantId: str = Query(...)):
    print("Fetching uploaded file lists")
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    tenantFiles = []
    for item in upload_documents.values():
        if item.get('tenantId') == tenantId:
            if 'fileName' in item and not item.get('fileName', '').startswith('Q&A_input_'):
                tenantFiles.append(item)
            elif 'source' in item and item['source'].startswith('https'):
                tenantFiles.append(item)

    return {"data": tenantFiles}

############################# prompts #############################
def initializeVectorStore():
    """Initialize and configure vector store with embeddings
    Returns:
        Configured PineconeVectorStore instance
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0
)

def detect_request_type(keyword, vectorstore, tenantId):
    """Detect the type of request based on the keyword and metadata of the documents."""
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 5,
            "filter": {
                'tenantId': {'$eq': tenantId}
            }
        }
    )
    docs = retriever.get_relevant_documents(keyword)
    
    #  metadata to determine the type
    tenant_files = []
    for doc in docs:
        metadata = doc.metadata  
        if metadata.get('tenantId') == tenantId:
            if 'fileName' in metadata and not metadata.get('fileName', '').startswith('Q&A_input_'):
                tenant_files.append(metadata)
            elif 'source' in metadata and metadata['source'].startswith('https'):
                tenant_files.append(metadata)
    
    #  type based on metadata
    if any(metadata.get('fileName', '').startswith('Q&A_input_') for metadata in tenant_files):
        return "qa"
    elif any(metadata.get('source', '').startswith('https') for metadata in tenant_files):
        return "Link"
    else:
        return "Document"

@app.get("/api/artificial-intelligence/prompts")
async def unified_prompts_endpoint(
    tenantId: str = Query(...), 
    keyword: str = Query(...)
):
    """Unified endpoint to handle different types of prompts
    Args:
        tenantId: Identifier for the tenant
        keyword: Search keyword/prompt
    Returns:
        PlainText response with answer or error message
    """
    try:
        print(tenantId, keyword)
        vectorstore = initializeVectorStore()
        
        # the type of request
        request_type = detect_request_type(keyword, vectorstore, tenantId)
        
        if request_type == "Document":
            template = """Answer the question based only on the following context:
            {context}
            User Question:
            {question}
            If the user asks you, 'Who are you?' or 'What are you?' respond with: "I am an AI assistant."
            If the user greets you, respond with one of the following without searching information from knowledge base:
            - "Hello! How can I assist you today?"
            - "Hi there! What can I do for you?"
            - "Good day! How can I assist you?"
            - "Hey! What can I help you with?"
            - "Hi! How can I support you today?"
            - "Welcome! How may I help you?"
            - "Salutations! How can I assist you?"
            - "What's up? How can I help?"
            - "Good to see you! How can I assist?"
            Please provide a concise and accurate answer based on the context above. If the context does not contain information to answer the question, respond with: "I don't have the information you're looking for, please provide additional details."
            Make sure your answer is relevant and accurate to the question and does not repeat the question itself.
            In response give only answer but not question.
            """
            retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={
                    "k": 5,
                    "filter": {
                        'tenantId': {'$eq': tenantId}
                    }
                }
            )
            prompt = ChatPromptTemplate.from_template(template)
            rag_chain = (
                {"context": retriever, "question": RunnablePassthrough()}
                | prompt
                | llm
                | StrOutputParser())
            answer = rag_chain.invoke(keyword)
            return PlainTextResponse(answer)

        elif request_type == "Link":
            template = """Based on the provided web content, answer the following question:
            {context}
            User Question:
            {question}
            If the user asks you, 'Who are you?' or 'What are you?' respond with: "I am an AI assistant."
            If the user greets you, respond with one of the following without searching information from knowledge base:
            - "Hello! How can I assist you today?"
            - "Hi there! What can I do for you?"
            - "Good day! How can I assist you?"
            - "Hey! What can I help you with?"
            - "Hi! How can I support you today?"
            - "Welcome! How may I help you?"
            - "Salutations! How can I assist you?"
            - "What's up? How can I help?"
            - "Good to see you! How can I assist?"
             Instructions for answering:
            1. Extract relevant information directly from the web content
            2. If the question is about specific data or facts, cite them accurately
            3. If the content contains multiple relevant sections, synthesize them into a coherent answer
            4. For technical or complex topics, provide clear, simplified explanations
            5. If the information is not available in the context, respond with: "This information is not available in the source content. Please try a different question or provide more details."
            6. For greetings, use one of the greeting responses above
            7. If the user asks questions that are synonyms or rephrased versions of questions in URL links, provide relevant answers based on the intent of the question, not just the exact wording.
            8. When answering, focus on matching the intent of the user's question, even if the exact words are not present in the context. Use contextual understanding to infer the meaning and provide accurate responses.
            9. If the question is ambiguous or unclear, ask for clarification to ensure the response aligns with the user's intent.
            10. For numerical data or specific metrics (e.g., measurements, statistics, or specifications), provide the exact match from the context.
            Remember:- For example: The secondary sensor of the Google Pixel 9 with 48 MP ultra-wide-angle, f2.2 and field of view (FOV) is 125.8˚and
                                 The secondary sensor of the Galaxy S25 with 12 MP ultra-wide-angle, f/2.2 aperture and field of view (FOV) is 120˚ 
            """
            retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={
                    "k": 8,  
                    "filter": {
                        'tenantId': {'$eq': tenantId}
                    }
                }
            )
            docs = retriever.get_relevant_documents(keyword)
            corpus = [doc.page_content for doc in docs]
            tokenized_corpus = [word_tokenize(doc.lower()) for doc in corpus]
            bm25 = BM25Okapi(tokenized_corpus)
            tokenized_query = word_tokenize(keyword.lower())
            bm25_scores = bm25.get_scores(tokenized_query)
            reranked_pairs = list(zip(docs, bm25_scores))
            reranked_pairs.sort(key=lambda x: x[1], reverse=True)
            reranked_docs = [pair[0] for pair in reranked_pairs[:1]]
            prompt = ChatPromptTemplate.from_template(template)
            rag_chain = (
                {"context": lambda x: reranked_docs, "question": RunnablePassthrough()}
                | prompt
                | llm
                | StrOutputParser())
            answer = rag_chain.invoke(keyword)
            return PlainTextResponse(answer)

        elif request_type == "qa":
            template = """Answer the question based only on the following context:
            {context}
            User Question:
            {question}
            If the user asks you, 'Who are you?' or 'What are you?' respond with: "I am an AI assistant."
            If the user greets you, respond with one of the following without searching information from knowledge base:
            - "Hello! How can I assist you today?"
            - "Hi there! What can I do for you?"
            - "Good day! How can I assist you?"
            - "Hey! What can I help you with?"
            - "Hi! How can I support you today?"
            - "Welcome! How may I help you?"
            - "Salutations! How can I assist you?"
            - "What's up? How can I help?"
            - "Good to see you! How can I assist?"
            Please follow these rules:
            1. If the user's question EXACTLY matches a question in the context (case-insensitive), provide the corresponding answer.
            2. If there is no exact match, respond with: "Please provide additional details."
            Make sure your answer is relevant and accurate to the question and does not repeat the question itself.
            In response give only answer but not question.
            """
            retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={
                    "k": 1,  
                    "filter": {
                        'tenantId': {'$eq': tenantId}
                    }
                }
            )
            docs = retriever.get_relevant_documents(keyword)
            exact_match = False
            exact_answer = None
            for doc in docs:
                if "Question:" in doc.page_content and "Answer:" in doc.page_content:
                    qa_parts = doc.page_content.split("\n")
                    if len(qa_parts) >= 2:
                        stored_question = qa_parts[0].replace("Question:", "").strip()
                        stored_answer = qa_parts[1].replace("Answer:", "").strip()
                        if stored_question.lower() == keyword.lower():
                            exact_match = True
                            exact_answer = stored_answer
                            break
            if exact_match:
                return PlainTextResponse(exact_answer)
            # else:
            #     prompt = ChatPromptTemplate.from_template(template)
            #     rag_chain = (
            #         {"context": retriever, "question": RunnablePassthrough()}
            #         | prompt
            #         | llm
            #         | StrOutputParser())
            #     answer = rag_chain.invoke(keyword)
            #     return PlainTextResponse(answer)

        else:
            raise HTTPException(status_code=400, detail="Invalid type parameter. Use 'upload', 'links', or 'qa'.")

    except Exception as e:
        error_msg = f"Error processing your request: {str(e)}. Please contact support for assistance."
        print(f"Error in unified_prompts_endpoint: {str(e)}") 
        return PlainTextResponse(error_msg, status_code=500)

############################ Summarizer #################################################
class SummarizationRequest(BaseModel):
    caseId: str
    assignedTo: str
    patientName: str
    onBehalfOf: str
    currentCaseStatus: str
    summarizationType: str
    notes: List[Any]

@app.post("/api/artificial-intelligence/summarize")
async def summarize(request: SummarizationRequest):
    """Combine JSON upload and summary generation into one endpoint."""
    try:
        json_data = {"caseId": request.caseId, "notes": request.notes, "patientName": request.patientName, "assignedTo": request.assignedTo, "currentCaseStatus": request.currentCaseStatus}
        noteLength = sum(len(note['note']) for note in request.notes)
        maxTokenToUse = 500
        
        if(noteLength <= 10):
            maxTokenToUse=200
        
        
        json_doc = Document(page_content=json.dumps(json_data), metadata={'fileName': 'query_json'})
        split_docs = [json_doc]
        print(f"Processing {len(split_docs)} documents.")
        
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.0,
            max_tokens=maxTokenToUse,
            top_p=1.0,
        )

        full_summary_prompt_template = f"""
        "{{split_docs}}"

        Please summarize the following notes directly to me, the {request.onBehalfOf} in a structured format, focusing on key actions and communications. Include the following sections:
        
        1. Initial Request: Briefly explain the initial issue or request.
        2. Key Updates: Highlight major actions taken, communications, and progress in chronological order, including dates.
        3. Current Status: Summarize the present situation or outcome of the case.

        Do not expose any ids

        Complete the sentence at the end with a full stop.
        Give html format
        """

        unread_summary_prompt_template = f"""
        "{{split_docs}}"

        Please summarize the following case notes directly to me, the {request.onBehalfOf} in a paragraph format.
        
        Do not expose any ids
        Keep it simple and short.
        Complete the sentence at the end with a full stop.
        """


        prompt=""
        if(request.summarizationType == 'full'):
            prompt = full_summary_prompt_template
        else:
            prompt = unread_summary_prompt_template
        
        print(prompt)

        prompt = PromptTemplate.from_template(prompt)
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="split_docs")
        summary = stuff_chain.run(split_docs)

        sentences = summary
        print(sentences)

        # if len(sentences) > 1:
        #     summary = '.'.join(sentences[:-1]).strip()  
        # else:
        #     summary = sentences[0].strip()  

        # if not summary.endswith('.'):
        #     summary += '.'
        
        summary = sentences.replace("\n", " ")
        summary = summary.replace("```html", " ")
        return JSONResponse(content={
            "summary": summary
        })
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error during summarizing the documents: {str(e)}")

####################################### delete #########################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/delete", summary="Delete Documents", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """Delete documents matching prefix
    Args:
        request: Delete request containing prefix
    Returns:
        JSON response indicating deletion status
    Raises:
        HTTPException: When no matching documents found
    """
    prefix = request.prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    print(ids_to_delete)
    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")
    index.delete(ids=ids_to_delete)
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    if prefix in upload_documents:
        del upload_documents[prefix]
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}











if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
11. If the user asks any date related information, search the context for date and provide the  match.
                The date extraction should work with formats like:    
                - 2022/02/09
                - 04/03/2024
                - 01/07/2023
                - 2022, January 09
                - March 09, 2022
            Remember:- For example: The Samsung Galaxy S22 Ultra 5G was launch on 2022, February 09.


            5. If the information is not available in the context, respond with: "This information is not available in the source content. Please try a different question or provide more details."
